## 대전광역시 수소 충전소 우선 입지 선정

### Load Library

In [1]:
## 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
## 설치 후, 런타임 다시 시작
!pip install pycaret --ignore-installed llvmlite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 6.8 MB/s 
     |████████████████████████████████| 34.6 MB 1.2 MB/s 
     |████████████████████████████████| 24.2 MB 1.3 MB/s 
     |████████████████████████████████| 81 kB 11.3 MB/s 
     |████████████████████████████████| 6.8 MB 56.1 MB/s 
     |████████████████████████████████| 167 kB 69.7 MB/s 
     |████████████████████████████████| 292 kB 65.9 MB/s 
     |████████████████████████████████| 25.9 MB 1.3 MB/s 
     |████████████████████████████████| 2.0 MB 50.5 MB/s 
     |████████████████████████████████| 282 kB 75.9 MB/s 
     |████████████████████████████████| 133 kB 66.7 MB/s 
     |████████████████████████████████| 88 kB 6.5 MB/s 
     |████████████████████████████████| 636 kB 55.5 MB/s 
     |████████████████████████████████| 262 kB 58.6 MB/s 
     |████████████████████████████████| 3.3 MB 59.9 MB/s 
     |█████████████████████████████

In [2]:
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes
!pip install --upgrade contextily

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
import re
from tqdm import tqdm
from time import sleep
from shapely.geometry import Point, Polygon, LineString
from pycaret.classification import *

### 데이터 전처리

In [4]:
## 대전광역시 격자데이터
total_gid = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/격자 데이터/total_gid/total_gid.shp', encoding = 'utf-8')

total_gid.to_crs(epsg = 4326, inplace = True)

#### 1. 대전광역시 수소충전소 현황

In [ ]:
daejeon_suso = pd.read_excel('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/대전시 수소충전소 현황.xlsx', skiprows = 2,  skipfooter = 2,usecols = ['이름', '주소'])
daejeon_suso

,이름,주소
0,학하수소충전소,대전광역시 유성구 유성대로 437
1,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72
2,낭월수소버스충전소,대전광역시 동구 금산로 455
3,대전유성수소충전소,대전광역시 유성구 현충원로 168
4,하이넷자운대충전소,대전광역시 유성구 자운로 62
5,대전중도가스수소충전소,대전광역시 동구 대전로 295
6,하이넷신탄진휴게소수소충전소,대전광역시 대덕구 산업단지로 27


In [ ]:
## 데이터에 위치좌표(위경도) 포함되어있지 않아 직접 추가
lat = [36.335908, 36.406480, 36.267989, 36.359694, 36.393269, 36.294452, 36.428283]
lon = [127.316275, 127.422514, 127.477952, 127.290630, 127.342414, 127.460783, 127.417995]

daejeon_suso['위도'] = lat; daejeon_suso['경도'] = lon
daejeon_suso.head(3)

,이름,주소,위도,경도
0,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275
1,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514
2,낭월수소버스충전소,대전광역시 동구 금산로 455,36.267989,127.477952


In [ ]:
## GeoDataFrame으로 변환
daejeon_suso = gpd.GeoDataFrame(daejeon_suso, crs = 'epsg:5179',
                                geometry = gpd.points_from_xy(
                                    daejeon_suso.경도,
                                    daejeon_suso.위도
                                ))
daejeon_suso.head(3)

,이름,주소,위도,경도,geometry
0,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,POINT (127.316 36.336)
1,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,POINT (127.423 36.406)
2,낭월수소버스충전소,대전광역시 동구 금산로 455,36.267989,127.477952,POINT (127.478 36.268)


In [ ]:
## shp file로 저장
daejeon_suso.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 수소충전소 현황/대전 수소충전소 현황.shp', encoding = 'utf-8')

shp 파일을 QGIS에서 로드 후, 충전소의 POINT 좌표별 버퍼 생성(250m, 500m, 750m, 1000m) 후 교차되는 격자 추출하여 저장

#### 2. 대전광역시 LPG 충전소 현황

In [ ]:
shp_file = '/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/대전광역시_충전소 현황_공간정보_20211221/대전광역시 충전소 현황.shp'
lpg_data = gpd.read_file(shp_file)

# LPG가 포함되어 있는 충전소 추출하여 저장
daejeon_lpg = lpg_data[lpg_data['종류'].str.contains('LPG')]
daejeon_lpg.head(3)

,인덱스,업체명,셀프,종류,연락처,주소,구,동,상표,위도,경도,geometry
16,17,월평LPG충전소,None,LPG,042-522-9464,대전 서구 월드컵대로484번길 82-59,서구,월평동,E1,36.346565,127.352735,POINT (986785.703 1816583.698)
40,41,SK행복충전 대전가스충전소,None,LPG,042-523-5897,대전 서구 도산로 218,서구,변동,SK에너지,36.327455,127.380776,POINT (989299.220 1814460.541)
45,46,서정충전소,None,LPG,042-582-8131,대전 서구 계백로 1332,서구,정림동,E1,36.309304,127.370180,POINT (988345.444 1812448.526)


In [ ]:
## shp file로 저장
daejeon_lpg.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 LPG 충전소 현황/대전 LPG 충전소 현황.shp', encoding = 'utf-8')

shp 파일을 QGIS에서 로드 후, 충전소의 POINT 좌표별 버퍼 생성(250m, 500m, 750m, 1000m) 후 교차되는 격자 추출하여 저장

#### 3. 대전광역시 유치원, 초·중·고등학교 현황

In [ ]:
daejeon_school = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/대전광역시_학교정보.csv')
daejeon_school.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr


In [ ]:
## 위경도 좌표 코드
def geocoding(address) :
  geolocoder = Nominatim(user_agent = 'South korea', timeout = None)
  geo = geolocoder.geocode(address)
  crd = {'lat':str(geo.latitude), 'lng':str(geo.longitude)}

  return crd

In [ ]:
## 위경도 반환 위해 괄호 안 문자 제거
regex = '\([^)]+\)'
ad = [re.sub(regex, '', daejeon_school['주소'][i]) for i in range(len(daejeon_school))]
daejeon_school['주소2'] = ad
daejeon_school.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN,대전광역시 유성구 유성대로155번길 46-2
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr,대전광역시 서구 가수원로 26
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr,대전광역시 서구 벌곡로 1359


In [ ]:
## 위경도 반환
# 반환되지 않는 경우가 있어 오류 발생 시 x 대입
a=[]
for i in range(len(daejeon_school)) :
    try :
        b = list(dict.values(geocoding(daejeon_school['주소2'][i])))
        a.append(b)
    except AttributeError:
        a.append(["X","X"])

In [ ]:
daejeon_school['위경도' ] = a
daejeon_school.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2,위경도
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN,대전광역시 유성구 유성대로155번길 46-2,"[36.3098239, 127.3135382]"
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr,대전광역시 서구 가수원로 26,"[36.2981192, 127.35466716552901]"
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr,대전광역시 서구 벌곡로 1359,"[36.2310111, 127.310356]"


In [ ]:
## 위경도 열에서 위도/경도 추출하여 열 생성
daejeon_school['위도'] = [daejeon_school['위경도'][i][0] for i in range(len(daejeon_school))]
daejeon_school['경도'] = [daejeon_school['위경도'][i][1] for i in range(len(daejeon_school))]

In [ ]:
## 위경도 좌표 없는 행 갯수 확인
(daejeon_school['위도'] == 'X').sum()

22

In [ ]:
## 위경도 좌표 확인
daejeon_school['위경도'][1]
daejeon_school.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2,위경도,위도,경도
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN,대전광역시 유성구 유성대로155번길 46-2,"[36.3098239, 127.3135382]",36.3098239,127.3135382
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr,대전광역시 서구 가수원로 26,"[36.2981192, 127.35466716552901]",36.2981192,127.35466716552901
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr,대전광역시 서구 벌곡로 1359,"[36.2310111, 127.310356]",36.2310111,127.310356


- 동, 구 열 생성

In [ ]:
## 동 나타내는 열 생성 - 없는 행 있음
gul = r'\(([^)]+)'
dong = [re.findall(gul,daejeon_school['주소'][i]) for i in range(len(daejeon_school))]
daejeon_school['동'] = dong
daejeon_school.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2,위경도,위도,경도,동
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN,대전광역시 유성구 유성대로155번길 46-2,"[36.3098239, 127.3135382]",36.3098239,127.3135382,[교촌동]
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr,대전광역시 서구 가수원로 26,"[36.2981192, 127.35466716552901]",36.2981192,127.35466716552901,[가수원동]
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr,대전광역시 서구 벌곡로 1359,"[36.2310111, 127.310356]",36.2310111,127.310356,[가수원동]


In [ ]:
## 구 나타내는 열 생성
gu = [daejeon_school['주소2'][i].split(" ")[1] for i in range(len(daejeon_school))]
daejeon_school['구'] = gu

In [ ]:
## 위경도 추출이 잘 된 것, x인 것 따로 데이터 생성
daejeon_school_not = daejeon_school.loc[daejeon_school['위도'] == 'X', :]
daejeon_school_yes = daejeon_school.loc[daejeon_school['위도'] != 'X', :]
daejeon_school_yes.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2,위경도,위도,경도,동,구
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN,대전광역시 유성구 유성대로155번길 46-2,"[36.3098239, 127.3135382]",36.3098239,127.3135382,[교촌동],유성구
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr,대전광역시 서구 가수원로 26,"[36.2981192, 127.35466716552901]",36.2981192,127.35466716552901,[가수원동],서구
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr,대전광역시 서구 벌곡로 1359,"[36.2310111, 127.310356]",36.2310111,127.310356,[가수원동],서구


- 위경도가 없는 행에 대해서는, 검색을 통하여 직접 엑셀에서 추가하여 csv 파일로 저장

In [ ]:
## 엑셀에서 작업한 위경도 추가한 파일 불러오기
daejeon_school_not = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/Daejeon_school_not.csv')
daejeon_school_not.head(3)

,Unnamed: 0,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2,위경도,위도,경도,동,구
0,17,18,초,기성초등학교길헌분교장,대전광역시 서구 당고개길 48(평촌동),35425,042-583-8084,042-585-8094,서부교육청,공립,http://giseonges.djsch.kr,대전광역시 서구 당고개길 48,"['X', 'X']",36.241494,127.316317,['평촌동'],서구
1,23,24,유,꿈내리유치원,대전광역시 서구 원도안로 242번길 24 (도안동),35354,042-546-2100,042-546-9933,서부교육청,사립,NaN,대전광역시 서구 원도안로 242번길 24,"['X', 'X']",36.324149,127.346105,['도안동'],서구
2,108,109,유,대전대암초등학교병설유치원,대전광역시 동구 동부로55-61,34664,042-280-5290,042-285-0565,동부교육청,공립,NaN,대전광역시 동구 동부로55-61,"['X', 'X']",36.323407,127.457277,[],동구


In [ ]:
## 위경도 있는 데이터와 없는 데이터 하나로 결합하기
daejeon_school = pd.concat([daejeon_school_yes, daejeon_school_not])
daejeon_school.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2,위경도,위도,경도,동,구,Unnamed: 0
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN,대전광역시 유성구 유성대로155번길 46-2,"[36.3098239, 127.3135382]",36.3098239,127.3135382,[교촌동],유성구,NaN
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr,대전광역시 서구 가수원로 26,"[36.2981192, 127.35466716552901]",36.2981192,127.35466716552901,[가수원동],서구,NaN
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr,대전광역시 서구 벌곡로 1359,"[36.2310111, 127.310356]",36.2310111,127.310356,[가수원동],서구,NaN


In [ ]:
daejeon_school = gpd.GeoDataFrame(daejeon_school, crs = 'epsg:4326',
                                  geometry = gpd.points_from_xy(
                                      daejeon_school.경도,
                                      daejeon_school.위도))

daejeon_school.head(3)

,번호,구분,학교명,주소,우편번호,전화번호,팩스번호,관할교육청,공/사립,홈페이지,주소2,위경도,위도,경도,동,구,Unnamed: 0,geometry
0,1,유,BK꿈사랑유치원,대전광역시 유성구 유성대로155번길 46-2 (교촌동),34224.0,042-542-2537,042-545-2533,서부교육청,사립,NaN,대전광역시 유성구 유성대로155번길 46-2,"[36.3098239, 127.3135382]",36.3098239,127.3135382,[교촌동],유성구,NaN,POINT (127.31354 36.30982)
1,2,중,가수원중학교,대전광역시 서구 가수원로 26 (가수원동),35389.0,042-541-0145,042-541-6623,서부교육청,공립,http://gswms.djsch.kr,대전광역시 서구 가수원로 26,"[36.2981192, 127.35466716552901]",36.2981192,127.35466716552901,[가수원동],서구,NaN,POINT (127.35467 36.29812)
2,3,초,가수원초등학교,대전광역시 서구 벌곡로 1359 (가수원동),35387.0,042-541-1337,042-541-6415,서부교육청,공립,http://gasuwones.djsch.kr,대전광역시 서구 벌곡로 1359,"[36.2310111, 127.310356]",36.2310111,127.310356,[가수원동],서구,NaN,POINT (127.31036 36.23101)


- 그리드의 Polygon 좌표 내에 학교 데이터의 Point 좌표가 포함되면 1, 포함되지 않으면 0으로 대입

In [ ]:
## 인덱스 재정렬
daejeon_school.reset_index(drop = True, inplace = True)

In [ ]:
## within으로 학교가 존재하는 격자 뽑기
f = lambda x: daejeon_school['geometry'].within(x)
result = total_gid['geometry'].apply(f)
yes_school_grid = total_gid[(result == True).any(axis = 1)]

In [ ]:
## 학교가 존재하는 격자에 1, 존재하지 않는 격자에 0 대입
a = []
for i in tqdm(range(len(total_gid))):
  if total_gid.iloc[[i]].index in list(yes_school_grid.index):
    school = 1
    a.append(school)
  else : 
    school = 0
    a.append(school)

100%|██████████| 54912/54912 [11:11<00:00, 81.73it/s]


In [ ]:
## 데이터 확인
total_gid['school'] = a
total_gid.head(3)

,gid,area,population,geometry,business_new,school
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0.4,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",1.0,0
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",1.0,0


In [ ]:
## shp 파일로 저장
total_gid.to_file("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 학교데이터/대전 학교데이터.shp", encoding = "utf-8")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


##### 버퍼 생성하여 가중치 부여하기

- 단순히 Point가 있는 그리드만 보기에는, 주변 그리드의 영향력까지 폭넓게 살펴봐야 한다고 생각하여 50m, 100m, 150m, 200m 반경을 주어 각 0.8, 0.6, 0.4, 0.2의 가중치를 부여해줌

QGIS 사용하여 Point 기준으로 버퍼를 생성하여 교차되는 그리드 추출

In [ ]:
## grid ID 확인
total_gid.head(3)

,gid,area,population,geometry,business_new,school
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0.4,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",1.0,0
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",1.0,0


In [ ]:
## 학교 point 버퍼별로 포함되는 데이터 불러오기
point = total_gid[total_gid['school'] == 1][['gid']]
buffer_50 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/학교 가중치/학교_50m.csv")
buffer_100 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/학교 가중치/학교_100m.csv")
buffer_150 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/학교 가중치/학교_150m.csv")
buffer_200 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/학교 가중치/학교_200m.csv")

In [ ]:
## 버퍼 50에 해당하는 그리드에서 point에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_50_total = []
for i in tqdm(range(len(buffer_50))) :
    if buffer_50['gid'][i] not in list(point['gid']) :
        a = buffer_50['gid'][i]
        buffer_50_total.append(a)

buffer_50_total = pd.DataFrame({'gid':buffer_50_total})
buffer_50_total.head(3)

100%|██████████| 2140/2140 [00:00<00:00, 21840.55it/s]


,gid
0,다바894175
1,다바894175
2,다바894175


In [ ]:
## 버퍼 100에 해당하는 그리드에서 50에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_100_total = []
for i in tqdm(range(len(buffer_100))) :
    if buffer_100['gid'][i] not in list(buffer_50['gid']) :
        a = buffer_100['gid'][i]
        buffer_100_total.append(a)

buffer_100_total = pd.DataFrame({'gid':buffer_100_total})
buffer_100_total.head(3)

100%|██████████| 4537/4537 [00:00<00:00, 4850.49it/s]


,gid
0,다바962121
1,다바962121
2,다바881183


In [ ]:
## 버퍼 150에 해당하는 그리드에서 100에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_150_total = []
for i in tqdm(range(len(buffer_150))) :
    if buffer_150['gid'][i] not in list(buffer_100['gid']) :
        a = buffer_150['gid'][i]
        buffer_150_total.append(a)

buffer_150_total = pd.DataFrame({'gid':buffer_150_total})
buffer_150_total.head(3)

100%|██████████| 7864/7864 [00:03<00:00, 2413.62it/s]


,gid
0,다바901198
1,다바895167
2,다바894162


In [ ]:
## 버퍼 200에 해당하는 그리드에서 150에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_200_total = []
for i in tqdm(range(len(buffer_200))):
    if buffer_200['gid'][i] not in list(buffer_150['gid']):
        a = buffer_200['gid'][i]
        buffer_200_total.append(a)

buffer_200_total = pd.DataFrame({'gid':buffer_200_total})
buffer_200_total.head(3)

100%|██████████| 12021/12021 [00:08<00:00, 1449.17it/s]


,gid
0,다바943169
1,다바898166
2,다바898166


In [ ]:
##  스쿨의 포인트 기준 버퍼별 포함되는 그리드 값에 따라 가중치 부여
new_school = []
for i in tqdm(range(len(total_gid))) :
    if total_gid['gid'][i] in list(buffer_50_total['gid']) :
        a = 0.8
        new_school.append(a)
    elif total_gid['gid'][i] in list(point['gid']) :
        a = 1
        new_school.append(a)
    elif total_gid['gid'][i] in list(buffer_100_total['gid']) :
        a = 0.6
        new_school.append(a)
    elif total_gid['gid'][i] in list(buffer_150_total['gid']) :
        a = 0.4
        new_school.append(a)
    elif total_gid['gid'][i] in list(buffer_200_total['gid']) :
        a = 0.2
        new_school.append(a)
    else :
        a = 0
        new_school.append(a)


100%|██████████| 54912/54912 [00:54<00:00, 1004.69it/s]


In [ ]:
## 학교point와 그 반경에 가중치를 부여한 것을 열에 추가로 기입 -  school_new column
total_gid['school_new'] = new_school

In [ ]:
## csv 파일로 저장
total_gid.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 학교데이터/대전 학교 가중치.csv', encoding = 'utf-8')

#### 4. 대전광역시 노드, 링크

##### 도로 링크

In [ ]:
link = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/NODELINKDATA2/MOCT_LINK.shp')

## 도로 링크 데이터 중, 대전광역시 데이터만 추출
link = link[link['LINK_ID'].str.contains('183|184|185|186|187')]
link.head(3)

,LINK_ID,F_NODE,T_NODE,LANES,ROAD_RANK,ROAD_TYPE,ROAD_NO,ROAD_NAME,ROAD_USE,MULTI_LINK,CONNECT,MAX_SPD,REST_VEH,REST_W,REST_H,LENGTH,REMARK,geometry
0,1860689001,1865963801,1865963701,2,107,000,-,국제과학로,0,0,000,30,0,0.0,0,414.856758,None,"LINESTRING (232063.149 427951.664, 231856.525 ..."
1,1860688901,1865963701,1865963801,2,107,000,-,국제과학로,0,0,000,30,0,0.0,0,414.211972,None,"LINESTRING (231788.078 427642.678, 231824.456 ..."
14,1860643301,1865950101,1865950201,2,107,000,-,국제과학9로,0,0,000,60,0,0.0,0,171.235401,None,"LINESTRING (232237.149 429565.021, 232406.601 ..."


In [ ]:
## 좌표계 4326으로 변경
link.to_crs(epsg = 4326, inplace = True)
print(link.crs)

epsg:4326


In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    a = link['geometry'].intersects(gid).sum()
    cnt_list.append(a)

100%|██████████| 54912/54912 [49:16<00:00, 18.57it/s]


In [ ]:
cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

link = pd.DataFrame()
link['gid'] = gid
link['cnt'] = cnt_l

In [ ]:
## 도로 링크의 갯수별로 group화 후, 갯수 카운트
link.groupby(['cnt']).count()

,gid
cnt,
0,41389
1,1709
2,7234
3,790
4,1195
5,366
6,1050
7,181
8,521


In [ ]:
## 대전광역시 격자 데이터와, 도로 링크 데이터 merge
grid1 = total_gid.merge(link, on = 'gid', how='left') 
grid1

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,0
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,2
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,3
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,1
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,2
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv, shp 파일로 저장
grid1.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/도로 노드링크 데이터/도로 링크 데이터.csv', encoding = 'utf-8')
grid1.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/도로 노드링크 데이터/도로 링크 데이터.shp', encoding = 'utf-8')

##### 도로 노드

In [ ]:
node = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/NODELINKDATA2/MOCT_NODE.shp', encoding = 'cp949')

## 도로 emd 데이터 중, 대전광역시 데이터만 추출
node = node[node['NODE_ID'].str.contains('183|184|185|186|187')]
node.head(3)

,NODE_ID,NODE_TYPE,NODE_NAME,TURN_P,REMARK,geometry
1,1865949201,101,둔곡국가산업단지,0,None,POINT (232175.564 428286.284)
2,3280218301,101,-,0,교차로명없음,POINT (262099.284 260200.964)
5,1865963901,101,-,0,교차로명없음,POINT (232151.920 428147.833)


In [ ]:
## 좌표계 4326 변경
node.to_crs(epsg = 4326, inplace = True)
print(node.crs)

epsg:4326


In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    a = node['geometry'].within(gid).sum()
    cnt_list.append(a)

100%|██████████| 54912/54912 [18:27<00:00, 49.60it/s]


In [ ]:
cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

node = pd.DataFrame()
node['gid'] = gid
node['cnt'] = cnt_l

In [ ]:
## 도로 노드 갯수별로 group화 한 후, 갯수 카운트
node.groupby(['cnt']).count()

,gid
cnt,
0,51909
1,2582
2,354
3,48
4,17
5,2


In [ ]:
## 대전광역시 격자 데이터와, 도로 노드 데이터 merge
grid2 = total_gid.merge(node, on = 'gid', how='left')
grid2

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,0
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,0
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,0
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,0
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv 파일로 저장
grid2.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/도로 노드링크 데이터/도로 노드 데이터.csv', encoding = 'utf-8-sig')
grid2.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/도로 노드링크 데이터/도로 노드 데이터.shp', encoding = 'utf-8')

#### 5. 대전광역시 동별 화재 데이터

In [ ]:
daejeon_fire = pd.read_excel('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/동별 화재발생현황(2021-2022.7.).xlsx', skiprows = 3)
daejeon_fire.head(3)

,Unnamed: 0,연번,시군구,읍면동,화재건수,발생비율,순위,Unnamed: 7,Unnamed: 8,시군구.1,읍면동.1,화재건수.1,발생비율.1,순위.1,Unnamed: 14,Unnamed: 15,총건수,발생비율.2,순위.2
0,NaN,1,동구,가양동,20,0.020429,6,NaN,NaN,동구,가양동,7,0.011570,21,NaN,NaN,27,0.017045,9
1,NaN,2,동구,가오동,8,0.008172,30,NaN,NaN,동구,가오동,2,0.003306,79,NaN,NaN,10,0.006313,50
2,NaN,3,동구,구도동,1,0.001021,107,NaN,NaN,동구,구도동,1,0.001653,104,NaN,NaN,2,0.001263,111


In [ ]:
daejeon_fire = daejeon_fire[['시군구', '읍면동', '총건수', '발생비율.2', '순위.2']]
daejeon_fire = daejeon_fire.rename(columns = {'발생비율.2':'발생비율', '순위.2' : '순위'})
daejeon_fire.head(3)

,시군구,읍면동,총건수,발생비율,순위
0,동구,가양동,27,0.017045,9
1,동구,가오동,10,0.006313,50
2,동구,구도동,2,0.001263,111


In [ ]:
## 행정동 shp 파일 불러오기 (시군구)
shp_file = '/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/격자 데이터/SIG_20220324/sig.shp'
sig = gpd.read_file(shp_file, encoding = 'cp949')

## 대전만 뽑기 (시군구 코드 30)
cnd = sig['SIG_CD'].str.slice(0,2) == '30'
sig_30 = sig[cnd]
sig_30 = sig_30[['SIG_CD', 'SIG_KOR_NM']]
sig_30.head(3)

,SIG_CD,SIG_KOR_NM
64,30110,동구
65,30140,중구
66,30170,서구


In [ ]:
## 행정동 shp 파일 불러오기 (읍면동)
shp_file = '/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/격자 데이터/EMD_20220324/emd.shp'
emd = gpd.read_file(shp_file, encoding = 'cp949')

## 대전만 뽑기 (읍면동 코드 30)
emd1 = emd['EMD_CD'].str.slice(0,2) == '30'
emd_30 = emd[emd1]

## 시군구 행정동 파일과 행정동 코드를 기준으로 결합하기 위함
emd_30['SIG_EMD'] = emd['EMD_CD'].str.slice(0, 5)
emd_30.head(3)

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry,SIG_EMD
1221,30110101,Won-dong,원동,"POLYGON ((994102.259 1814631.711, 994103.803 1...",30110
1222,30110102,In-dong,인동,"POLYGON ((994532.876 1814101.870, 994523.422 1...",30110
1223,30110103,Hyo-dong,효동,"POLYGON ((994753.444 1813546.801, 994759.890 1...",30110


In [ ]:
## 시군구 행정동 파일과 읍면동 행정동 파일 merge
sig_30_gu = emd_30.merge(sig_30, left_on = 'SIG_EMD', right_on = 'SIG_CD', how = 'left')
sig_30_gu = sig_30_gu[['EMD_CD', 'EMD_ENG_NM', 'EMD_KOR_NM', 'geometry', 'SIG_KOR_NM']]
sig_30_gu.head(3)

,EMD_CD,EMD_ENG_NM,EMD_KOR_NM,geometry,SIG_KOR_NM
0,30110101,Won-dong,원동,"POLYGON ((994102.259 1814631.711, 994103.803 1...",동구
1,30110102,In-dong,인동,"POLYGON ((994532.876 1814101.870, 994523.422 1...",동구
2,30110103,Hyo-dong,효동,"POLYGON ((994753.444 1813546.801, 994759.890 1...",동구


In [ ]:
## 대전광역시 화재 발생 데이터와 행정동 파일 결합
sig_30_tb = sig_30_gu.merge(daejeon_fire, left_on = ['SIG_KOR_NM', 'EMD_KOR_NM'], right_on = ['시군구', '읍면동'], how = 'left')

## 결측값 0으로 채우기
sig_30_tb = sig_30_tb.fillna(0)
sig_30_tb.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 177 entries, 0 to 176
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   EMD_CD      177 non-null    object  
 1   EMD_ENG_NM  177 non-null    object  
 2   EMD_KOR_NM  177 non-null    object  
 3   geometry    177 non-null    geometry
 4   SIG_KOR_NM  177 non-null    object  
 5   시군구         177 non-null    object  
 6   읍면동         177 non-null    object  
 7   총건수         177 non-null    float64 
 8   발생비율        177 non-null    float64 
 9   순위          177 non-null    float64 
dtypes: float64(3), geometry(1), object(6)
memory usage: 15.2+ KB


In [ ]:
## 필요한 열만 추출하여 최종 데이터 생성
daejeon_fire = sig_30_tb[['EMD_CD', 'SIG_KOR_NM', 'EMD_KOR_NM', '총건수', 'geometry']]
daejeon_fire.head(3)

,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry
0,30110101,동구,원동,3.0,"POLYGON ((994102.259 1814631.711, 994103.803 1..."
1,30110102,동구,인동,3.0,"POLYGON ((994532.876 1814101.870, 994523.422 1..."
2,30110103,동구,효동,2.0,"POLYGON ((994753.444 1813546.801, 994759.890 1..."


In [ ]:
## shp file로 저장
daejeon_fire.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 화재 현황/대전 화재 현황.shp', encoding = 'utf-8')

전처리 후, QGIS 이용하여 각 격자의 중심점 추출

중심점 추출 후, 대전시 격자 데이터와 교차지점 이용하여 결합

In [ ]:
## 중심점만 추출된 shp 파일 불러오기
total_gid_point = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/total_gid_point1/total_gid_point1.shp', encoding = 'utf-8')

## 좌표계 변경
total_gid_point.to_crs(epsg = 4326, inplace = True)

## 건수별로 카운트
total_gid_point = total_gid_point.sort_values(by = '총건수', ascending = False)
total_gid_point = total_gid_point.reset_index(drop = True)
total_gid_point.head(3)

,gid,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry
0,다바884169,6125.49,452.0,30170111,서구,갈마동,64.0,POINT (127.37128 36.34989)
1,다바880166,388.45,297.0,30170111,서구,갈마동,64.0,POINT (127.36682 36.34718)
2,다바883165,418.27,241.0,30170111,서구,갈마동,64.0,POINT (127.37017 36.34628)


##### 중복행 확인

In [ ]:
## 전체적인 중복행 확인
total_gid_point[total_gid_point.duplicated(subset = ['gid', 'geometry'])]

,gid,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry
47853,다바941094,NaN,NaN,30140121,중구,구완동,0.0,POINT (127.43486 36.28232)
48113,다바944078,NaN,NaN,30140121,중구,구완동,0.0,POINT (127.43821 36.26790)
52288,다바944106,NaN,NaN,30140108,중구,호동,0.0,POINT (127.43819 36.29314)


In [ ]:
total_gid_point[total_gid_point['gid'].str.contains('다바944106')]

,gid,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry
27671,다바944106,NaN,NaN,30110138,동구,이사동,3.0,POINT (127.43819 36.29314)
52288,다바944106,NaN,NaN,30140108,중구,호동,0.0,POINT (127.43819 36.29314)


In [ ]:
total_gid_point[total_gid_point['gid'].str.contains('다바944078')]

,gid,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry
21556,다바944078,NaN,NaN,30110141,동구,소호동,4.0,POINT (127.43821 36.26790)
48113,다바944078,NaN,NaN,30140121,중구,구완동,0.0,POINT (127.43821 36.26790)


In [ ]:
total_gid_point[total_gid_point['gid'].str.contains('다바941094')]

,gid,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry
27782,다바941094,NaN,NaN,30110138,동구,이사동,3.0,POINT (127.43486 36.28232)
47853,다바941094,NaN,NaN,30140121,중구,구완동,0.0,POINT (127.43486 36.28232)


In [ ]:
## 확인한 중복행 마지막 값 남기고 모두 제거
total_gid_point = total_gid_point.drop_duplicates(['gid', 'geometry'], keep = 'last')
total_gid_point

,gid,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry
0,다바884169,6125.49,452.0,30170111,서구,갈마동,64.0,POINT (127.37128 36.34989)
1,다바880166,388.45,297.0,30170111,서구,갈마동,64.0,POINT (127.36682 36.34718)
2,다바883165,418.27,241.0,30170111,서구,갈마동,64.0,POINT (127.37017 36.34628)
3,다바889160,415.86,162.0,30170111,서구,갈마동,64.0,POINT (127.37686 36.34178)
4,다바883176,413.84,94.0,30170111,서구,갈마동,64.0,POINT (127.37015 36.35620)
...,...,...,...,...,...,...,...,...
54910,다바784137,NaN,NaN,None,None,None,NaN,POINT (127.25992 36.32087)
54911,라바039220,NaN,NaN,None,None,None,NaN,POINT (127.54405 36.39593)
54912,다바813034,NaN,NaN,None,None,None,NaN,POINT (127.29247 36.22807)
54913,다바883080,NaN,NaN,None,None,None,NaN,POINT (127.37030 36.26965)


In [ ]:
## 건수별 그룹화하여 확인
total_gid_point.groupby('총건수').count()

,gid,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,geometry
총건수,,,,,,,
0.0,9998,621,544,9998,9998,9998,9998
1.0,8055,684,604,8055,8055,8055,8055
2.0,8108,931,739,8108,8108,8108,8108
3.0,3830,533,473,3830,3830,3830,3830
4.0,4360,900,742,4360,4360,4360,4360
5.0,1708,738,572,1708,1708,1708,1708
6.0,754,137,129,754,754,754,754
7.0,1204,478,348,1204,1204,1204,1204
8.0,1945,611,523,1945,1945,1945,1945


In [ ]:
total_gid_point.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 화재데이터/대전 화재데이터.shp', encoding = 'utf-8')
total_gid_point.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 화재데이터/대전 화재데이터.csv', encoding = 'utf-8')

##### 구/동별 gid(격자) 개수 카운트

In [ ]:
## 구와 동별로 그룹화하여 카운트
a = total_gid_point.groupby(['SIG_KOR_NM','EMD_KOR_NM']).count()

## 인덱스 리셋해주기
a = a.reset_index()
a = a.reset_index(drop = True)

## 필요한 column만 추출
a = a[['SIG_KOR_NM', 'EMD_KOR_NM', 'gid']]
a

,SIG_KOR_NM,EMD_KOR_NM,gid
0,대덕구,갈전동,320
1,대덕구,대화동,318
2,대덕구,덕암동,124
3,대덕구,목상동,72
4,대덕구,문평동,221
...,...,...,...
172,중구,정생동,691
173,중구,중촌동,119
174,중구,침산동,304
175,중구,태평동,166


In [ ]:
## csv 파일로 저장
a.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 화재데이터/법정동_gid개수.csv', encoding = 'utf-8', index = False)

##### 구/읍면동 기준 gid 개수 합치기

In [ ]:
total_gid_point = total_gid_point.merge(a, on = ['SIG_KOR_NM', 'EMD_KOR_NM'], how = 'left')
total_gid_point.head(3)

,gid_x,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry,gid_y
0,다바884169,6125.49,452.0,30170111,서구,갈마동,64.0,POINT (127.37128 36.34989),221.0
1,다바880166,388.45,297.0,30170111,서구,갈마동,64.0,POINT (127.36682 36.34718),221.0
2,다바883165,418.27,241.0,30170111,서구,갈마동,64.0,POINT (127.37017 36.34628),221.0


##### 동별 화재 발생 건수 / gid 개수

In [ ]:
total_gid_point['격자별화재건수'] = total_gid_point['총건수']/total_gid_point['gid_y']
total_gid_point.head(3)

,gid_x,area,population,EMD_CD,SIG_KOR_NM,EMD_KOR_NM,총건수,geometry,gid_y,격자별화재건수
0,다바884169,6125.49,452.0,30170111,서구,갈마동,64.0,POINT (127.37128 36.34989),221.0,0.289593
1,다바880166,388.45,297.0,30170111,서구,갈마동,64.0,POINT (127.36682 36.34718),221.0,0.289593
2,다바883165,418.27,241.0,30170111,서구,갈마동,64.0,POINT (127.37017 36.34628),221.0,0.289593


In [ ]:
## csv 파일로 저장
total_gid_point.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 화재데이터/대전격자별화재건수.csv', encoding='utf-8', index = False)

#### 6. 대전광역시 상가 정보

In [ ]:
## 파일 불러오기
data = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/소상공인시장진흥공단_상가(상권)정보_대전_202206.csv')

In [ ]:
## GeoDataFrame으로 변환하기
daejeon_business = gpd.GeoDataFrame(data, crs = 'epsg:4326',
                                geometry = gpd.points_from_xy(
                                    data.경도,
                                    data.위도
                                ))
daejeon_business.head(3)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,geometry
0,22401931,스피드메이트,NaN,F,생활서비스,F14,자동차/이륜차,F14A01,자동차정비/카센타,NaN,...,NaN,대전광역시 유성구 관들4길 25,305509,34018.0,NaN,NaN,NaN,127.393241,36.424448,POINT (127.39324 36.42445)
1,24185362,국제종합기계부품총판,NaN,D,소매,D25,기타판매업,D25A15,농기구판매,G47512,...,써머스빌,대전광역시 서구 계백로 1426,302160,35400.0,NaN,NaN,NaN,127.379765,36.312625,POINT (127.37977 36.31262)
2,22742209,알파수학전문학원,NaN,R,학문/교육,R01,학원-보습교습입시,R01A01,학원-입시,P85501,...,선우빌딩,대전광역시 중구 선화서로 128-1,301824,34841.0,NaN,2.0,NaN,127.413968,36.334319,POINT (127.41397 36.33432)


In [ ]:
## 필요한 column만 추출하여 저장
daejeon_business = daejeon_business[['상가업소번호', '상호명','상권업종대분류명','상권업종중분류명','상권업종소분류명',
                                     '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명',
                                     '도로명주소', '경도', '위도', 'geometry']]
daejeon_business

,상가업소번호,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,도로명주소,경도,위도,geometry
0,22401931,스피드메이트,생활서비스,자동차/이륜차,자동차정비/카센타,30200,유성구,3020060000,관평동,3020014600,관평동,대전광역시 유성구 관들4길 25,127.393241,36.424448,POINT (127.39324 36.42445)
1,24185362,국제종합기계부품총판,소매,기타판매업,농기구판매,30170,서구,3017053000,도마2동,3017010300,도마동,대전광역시 서구 계백로 1426,127.379765,36.312625,POINT (127.37977 36.31262)
2,22742209,알파수학전문학원,학문/교육,학원-보습교습입시,학원-입시,30140,중구,3014053500,은행선화동,3014010200,선화동,대전광역시 중구 선화서로 128-1,127.413968,36.334319,POINT (127.41397 36.33432)
3,17174039,솔바람,음식,한식,한식/백반/한정식,30200,유성구,3020054600,노은1동,3020012000,지족동,대전광역시 유성구 은구비서로3번길 8-23,127.320105,36.373504,POINT (127.32010 36.37350)
4,17174056,BBQ치킨,음식,유흥주점,호프/맥주,30140,중구,3014057500,대흥동,3014010500,대흥동,대전광역시 중구 대흥로 158,127.429579,36.324517,POINT (127.42958 36.32452)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73031,18443684,에이스컨설팅,생활서비스,기타서비스업,사업경영상담,30170,서구,3017055000,용문동,3017010500,용문동,대전광역시 서구 괴정로165번길 43,127.391133,36.335728,POINT (127.39113 36.33573)
73032,18446061,무브,소매,의복의류,여성의류전문점,30170,서구,3017059000,가수원동,3017011500,도안동,대전광역시 서구 도안동로31번길 12-22,127.351335,36.308643,POINT (127.35133 36.30864)
73033,18465896,감성커피,음식,커피점/카페,커피전문점/카페/다방,30140,중구,3014053500,은행선화동,3014010200,선화동,대전광역시 중구 대종로593번길 58,127.415871,36.333121,POINT (127.41587 36.33312)
73034,18443384,이마트,소매,종합소매점,편의점,30170,서구,3017059000,가수원동,3017011500,도안동,대전광역시 서구 도안북로 88,127.337456,36.328863,POINT (127.33746 36.32886)


In [ ]:
## 상권의 분류별로 개수 카운트
daejeon_business[['상권업종대분류명','상가업소번호']].groupby(['상권업종대분류명']).count()

,상가업소번호
상권업종대분류명,
관광/여가/오락,1770
부동산,2707
생활서비스,14029
소매,24597
숙박,585
스포츠,854
음식,23311
학문/교육,5183


In [ ]:
## 상권 분류별로 나누어, 변수에 저장
a = daejeon_business[daejeon_business['상권업종대분류명'] == '관광/여가/오락']
b = daejeon_business[daejeon_business['상권업종대분류명'] == '부동산']
c = daejeon_business[daejeon_business['상권업종대분류명'] == '생활서비스']
d = daejeon_business[daejeon_business['상권업종대분류명'] == '소매']
e = daejeon_business[daejeon_business['상권업종대분류명'] == '숙박']
f = daejeon_business[daejeon_business['상권업종대분류명'] == '스포츠']
g = daejeon_business[daejeon_business['상권업종대분류명'] == '음식']
h = daejeon_business[daejeon_business['상권업종대분류명'] == '학문/교육']

In [ ]:
## 좌표계 4326으로 변경
a.to_crs(epsg = 4326, inplace = True)
b.to_crs(epsg = 4326, inplace = True)
c.to_crs(epsg = 4326, inplace = True)
d.to_crs(epsg = 4326, inplace = True)
e.to_crs(epsg = 4326, inplace = True)
f.to_crs(epsg = 4326, inplace = True)
g.to_crs(epsg = 4326, inplace = True)
h.to_crs(epsg = 4326, inplace = True)

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


##### 상권 분류별로 격자별 개수 카운트

###### 관광/여가/오락

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    a1 = a['geometry'].within(gid).sum()
    
    cnt_list.append(a1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

a = pd.DataFrame()
a['gid'] = gid
a['cnt'] = cnt_l

a.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [05:06<00:00, 179.16it/s]


,cnt,gid
0,0,53843
1,1,696
2,2,214
3,3,80
4,4,34
5,5,30
6,6,5
7,7,2
8,8,3
9,9,1


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(a, on = 'gid', how='left') 
grid

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,0
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,0
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,0
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,0
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/a.csv", encoding = "utf-8-sig")

###### 부동산

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    b1 = b['geometry'].within(gid).sum()
    
    cnt_list.append(b1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

b = pd.DataFrame()
b['gid'] = gid
b['cnt'] = cnt_l

b.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [07:32<00:00, 121.40it/s]


,cnt,gid
0,0,53208
1,1,1149
2,2,311
3,3,134
4,4,54
5,5,32
6,6,16
7,7,5
8,8,2
9,11,1


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(b, on = 'gid', how='left') 
grid

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,0
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,2
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,0
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,0
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/b.csv", encoding = "utf-8-sig")

###### 생활서비스

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    c1 = c['geometry'].within(gid).sum()
    
    cnt_list.append(c1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

c = pd.DataFrame()
c['gid'] = gid
c['cnt'] = cnt_l
 
c.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [38:37<00:00, 23.69it/s]


,cnt,gid
0,0,49940
1,1,1914
2,2,1114
3,3,690
4,4,426
5,5,266
6,6,185
7,7,112
8,8,87
9,9,51


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(c, on = 'gid', how='left') 
grid

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,2
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,4
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,1
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,8
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/c.csv", encoding = "utf-8-sig")

###### 소매

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    d1 = d['geometry'].within(gid).sum()
    
    cnt_list.append(d1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

d = pd.DataFrame()
d['gid'] = gid
d['cnt'] = cnt_l

d.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [1:07:15<00:00, 13.61it/s]


,cnt,gid
0,0,48910
1,1,2168
2,2,1124
3,3,745
4,4,462
...,...,...
64,177,1
65,185,1
66,191,1
67,213,1


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(d, on = 'gid', how='left') 
grid

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,78
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,9
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,3
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,70
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/d.csv", encoding = "utf-8-sig")

###### 숙박

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    e1 = e['geometry'].within(gid).sum()
    
    cnt_list.append(e1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

e = pd.DataFrame()
e['gid'] = gid
e['cnt'] = cnt_l

e.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [01:49<00:00, 499.74it/s]


,cnt,gid
0,0,54601
1,1,178
2,2,74
3,3,24
4,4,14
5,5,8
6,6,5
7,7,4
8,8,3
9,9,1


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(e, on = 'gid', how='left') 
grid

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,1
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,0
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,0
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,0
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/e.csv", encoding = "utf-8-sig")

###### 스포츠

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    f1 = f['geometry'].within(gid).sum()
    
    cnt_list.append(f1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

f = pd.DataFrame()
f['gid'] = gid
f['cnt'] = cnt_l

f.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [02:35<00:00, 353.96it/s]


,cnt,gid
0,0,54293
1,1,455
2,2,123
3,3,24
4,4,9
5,5,4
6,6,3
7,7,1


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(f, on = 'gid', how='left') 
grid

,gid,area,population,geometry,school,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0,0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0,1
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0,0
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0,0
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0,0
...,...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0,0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0,0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0,0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0,0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/f.csv", encoding = "utf-8-sig")

###### 음식

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    g1 = g['geometry'].within(gid).sum()
    
    cnt_list.append(g1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

g = pd.DataFrame()
g['gid'] = gid
g['cnt'] = cnt_l

g.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [1:18:54<00:00, 11.60it/s]


,cnt,gid
0,0,49952
1,1,1574
2,2,817
3,3,565
4,4,395
5,5,292
6,6,224
7,7,174
8,8,150
9,9,117


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(g, on = 'gid', how='left') 
grid

,gid,area,population,geometry,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",31
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",8
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",3
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",44
...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/g.csv", encoding = "utf-8-sig")

###### 학문/교육

In [ ]:
cnt_list = []
for i in tqdm(range(total_gid.shape[0])):
    gid = total_gid['geometry'].iloc[i]
    g1 = h['geometry'].within(gid).sum()
    
    cnt_list.append(g1)

cnt_l = list(cnt_list)
gid = list(total_gid['gid'])

h = pd.DataFrame()
h['gid'] = gid
h['cnt'] = cnt_l

h.groupby(['cnt'], as_index = False).count()

100%|██████████| 54912/54912 [20:23<00:00, 44.88it/s]


,cnt,gid
0,0,52737
1,1,1231
2,2,392
3,3,196
4,4,99
5,5,78
6,6,40
7,7,33
8,8,22
9,9,17


In [ ]:
## 격자 데이터와 gid 기준으로 결합
grid = total_gid.merge(h, on = 'gid', how='left') 
grid

,gid,area,population,geometry,cnt
0,다바922281,125401.97,NaN,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0
1,다바909251,118444.81,NaN,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",1
2,다바905261,104399.92,11.0,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0
3,다바894175,102749.66,NaN,"POLYGON ((127.38186 36.35486, 127.38186 36.355...",0
4,다바943169,95862.58,NaN,"POLYGON ((127.43647 36.34949, 127.43647 36.350...",0
...,...,...,...,...,...
54907,다바888214,NaN,NaN,"POLYGON ((127.37511 36.39001, 127.37511 36.390...",0
54908,다바827236,NaN,NaN,"POLYGON ((127.30705 36.40975, 127.30704 36.410...",0
54909,다바827134,NaN,NaN,"POLYGON ((127.30727 36.31780, 127.30727 36.318...",0
54910,다바898317,NaN,NaN,"POLYGON ((127.38613 36.48288, 127.38613 36.483...",0


In [ ]:
## csv 파일로 저장
grid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/h.csv", encoding = "utf-8-sig")

##### 데이터 병합

In [ ]:
## 격자별 개수 카운트하여 저장한 데이터 불러오기
a = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/a.csv')
b = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/b.csv')
c = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/c.csv')
d = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/d.csv')
e = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/e.csv')
f = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/f.csv')
g = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/g.csv')
h = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/h.csv')

In [ ]:
## 필요한 column만 선택 후, 열 이름 변경하기
a = a[['gid','cnt']]
b = b[['gid','cnt']]
c = c[['gid','cnt']]
d = d[['gid','cnt']]
e = e[['gid','cnt']]
f = f[['gid','cnt']]
g = g[['gid','cnt']]
h = h[['gid','cnt']]

a = a.rename(columns = {'cnt':'관광/여가/오락'})
b = b.rename(columns = {'cnt':'부동산'})
c = c.rename(columns = {'cnt':'생활서비스'})
d = d.rename(columns = {'cnt':'소매'})
e = e.rename(columns = {'cnt':'숙박'})
f = f.rename(columns = {'cnt':'스포츠'})
g = g.rename(columns = {'cnt':'음식'})
h = h.rename(columns = {'cnt':'학문/교육'})

In [ ]:
## 상권 분류 데이터 gid 기준으로 하나로 병합
grid = a.merge(b, on = 'gid', how='left')
grid = grid.merge(c, on = 'gid', how='left') 
grid = grid.merge(d, on = 'gid', how='left') 
grid = grid.merge(e, on = 'gid', how='left') 
grid = grid.merge(f, on = 'gid', how='left') 
grid = grid.merge(g, on = 'gid', how='left') 
grid = grid.merge(h, on = 'gid', how='left') 
grid.head(3)

,gid,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육
0,다바922281,0,0,0,0,0,0,0,0
1,다바909251,0,0,2,78,1,1,31,1
2,다바905261,0,2,4,9,0,0,8,0


In [ ]:
## csv 파일로 저장
grid.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/대전 상권 분류.csv', encoding = 'utf-8')

##### 버퍼 생성하여 가중치 부여하기

- 단순히 Point가 있는 그리드만 보기에는, 주변 그리드의 영향력까지 폭넓게 살펴봐야 한다고 생각하여 50m, 100m, 150m, 200m 반경을 주어 각 0.8, 0.6, 0.4, 0.2의 가중치를 부여해줌

QGIS 사용하여 Point 기준으로 버퍼를 생성하여 교차되는 그리드 추출

In [ ]:
point = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/상권 가중치/상권_point.csv', encoding = 'cp949')
point = pd.merge(point, total_gid, on = 'gid', how = 'inner')[['gid']]
point.head(3)

,gid
0,다바903123
1,다바927133
2,다바900127


In [ ]:
## 상권 point 버퍼반경별로 겹치는 그리드 id 데이터 불러오기
buffer_50 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/상권 가중치/상가_50m.csv")
buffer_100 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/상권 가중치/상가_100m.csv")
buffer_150 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/상권 가중치/상가_150m.csv")
buffer_200 = pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/상권 가중치/상가_200m.csv")

In [ ]:
## 중복 값 제거
point = point[['gid']].drop_duplicates().reset_index(drop = True)
buffer_50 = buffer_50[['gid']].drop_duplicates().reset_index(drop = True)
buffer_100 = buffer_100[['gid']].drop_duplicates().reset_index(drop = True)
buffer_150 = buffer_150[['gid']].drop_duplicates().reset_index(drop = True)
buffer_200 = buffer_200[['gid']].drop_duplicates().reset_index(drop = True)

In [ ]:
## 버퍼 50에 해당하는 그리드에서 point에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_50_total=[]
for i in tqdm(range(len(buffer_50))) :
    if buffer_50['gid'][i] not in list(point['gid']) :
        a = buffer_50['gid'][i]
        buffer_50_total.append(a)

buffer_50_total = pd.DataFrame({'gid':buffer_50_total})
buffer_50_total.head(3)

100%|██████████| 12987/12987 [00:10<00:00, 1247.20it/s]


,gid
0,다바882180
1,다바881183
2,다바893262


In [ ]:
## 버퍼 100에 해당하는 그리드에서 50에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_100_total = []
for i in tqdm(range(len(buffer_100))) :
    if buffer_100['gid'][i] not in list(buffer_50['gid']) :
        a = buffer_100['gid'][i]
        buffer_100_total.append(a)

buffer_100_total = pd.DataFrame({'gid':buffer_100_total})
buffer_100_total

100%|██████████| 16681/16681 [00:19<00:00, 845.42it/s]


,gid
0,다바858121
1,다바889116
2,다바918276
3,다바896197
4,다바908275
...,...
3689,다바915246
3690,다바874198
3691,다바822167
3692,다바875209


In [ ]:
## 버퍼 150에 해당하는 그리드에서 100에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_150_total = []
for i in tqdm(range(len(buffer_150))) :
    if buffer_150['gid'][i] not in list(buffer_100['gid']) :
        a = buffer_150['gid'][i]
        buffer_150_total.append(a)

buffer_150_total = pd.DataFrame({'gid':buffer_150_total})
buffer_150_total.head(3)

100%|██████████| 19974/19974 [00:31<00:00, 626.32it/s]


,gid
0,다바922281
1,다바915269
2,다바856188


In [ ]:
## 버퍼 200에 해당하는 그리드에서 150에 해당하는 그리드 제거하기 (점수 다르게 부여하기 위해서)
buffer_200_total = []
for i in tqdm(range(len(buffer_200))) :
    if buffer_200['gid'][i] not in list(buffer_150['gid']) :
        a = buffer_200['gid'][i]
        buffer_200_total.append(a)

buffer_200_total = pd.DataFrame({'gid':buffer_200_total})
buffer_200_total.head(3)

100%|██████████| 23076/23076 [00:43<00:00, 528.55it/s]


,gid
0,다바893199
1,다바976081
2,다바861119


In [ ]:
##  상권의 포인트 기준 버퍼반경별 포함되는 그리드 값에 따라 가중치 부여
new_business = []
for i in tqdm(range(len(total_gid))) :
    if total_gid['gid'][i] in list(buffer_50_total['gid']) :
        a = 0.8
        new_business.append(a)
    elif total_gid['gid'][i] in list(point['gid']) :
        a = 1
        new_business.append(a)
    elif total_gid['gid'][i] in list(buffer_100_total['gid']) :
        a = 0.6
        new_business.append(a)
    elif total_gid['gid'][i] in list(buffer_150_total['gid']) :
        a = 0.4
        new_business.append(a)
    elif total_gid['gid'][i] in list(buffer_200_total['gid']) :
        a = 0.2
        new_business.append(a)
    else :
        a = 0
        new_business.append(a)

100%|██████████| 54912/54912 [02:15<00:00, 405.32it/s]


In [ ]:
## 상가point와 그 반경에 가중치를 부여한 것을 열에 추가로 기입 -  business_new column
total_gid['business_new'] = new_business

In [ ]:
## csv 파일로 저장
total_gid.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/대전 상권 가중치.csv', encoding = 'utf-8')

#### 7. 대전광역시 차량 등록현황

In [ ]:
## 차량 등록 데이터 불러오기
daejeon_car = pd.read_excel('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_전/대전광역시 행정동별 차량등록현황.xlsx')
daejeon_car.head(3)

,구,행정동,계
0,대덕구,오정동,9171
1,대덕구,대화동,5762
2,대덕구,회덕동,10570


In [ ]:
## 행정구역 경계 데이터 불러오기
hjd = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/격자 데이터/2021년 센서스용 행정구역경계(읍면동)/bnd_dong_25_2021_2021_2Q.shp', encoding = 'cp949')
hjd.head(3)

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,geometry
0,20210630,2501053,효동,"POLYGON ((995219.809 1813575.224, 995224.704 1..."
1,20210630,2501055,판암1동,"POLYGON ((998324.140 1815195.386, 998303.010 1..."
2,20210630,2501056,판암2동,"POLYGON ((996271.740 1814199.434, 996279.403 1..."


In [ ]:
## 차량 데이터와 행정동 경계 데이터 동 기준으로 결합
car_hjd = hjd.merge(daejeon_car, left_on = 'ADM_DR_NM', right_on = '행정동')
car_hjd

,BASE_DATE,ADM_DR_CD,ADM_DR_NM,geometry,구,행정동,계
0,20210630,2501053,효동,"POLYGON ((995219.809 1813575.224, 995224.704 1...",동구,효동,10515
1,20210630,2501055,판암1동,"POLYGON ((998324.140 1815195.386, 998303.010 1...",동구,판암1동,5028
2,20210630,2501056,판암2동,"POLYGON ((996271.740 1814199.434, 996279.403 1...",동구,판암2동,2776
3,20210630,2501057,용운동,"POLYGON ((996692.308 1816363.836, 996715.821 1...",동구,용운동,9794
4,20210630,2501060,자양동,"POLYGON ((995842.464 1816397.515, 995842.820 1...",동구,자양동,3636
...,...,...,...,...,...,...,...
74,20210630,2505058,신탄진동,"POLYGON ((998333.118 1831051.084, 998569.441 1...",대덕구,신탄진동,7152
75,20210630,2505059,석봉동,"POLYGON ((993525.679 1828739.908, 993563.248 1...",대덕구,석봉동,8110
76,20210630,2505060,덕암동,"POLYGON ((992742.602 1827879.625, 992745.716 1...",대덕구,덕암동,9052
77,20210630,2505061,목상동,"POLYGON ((991707.326 1828583.879, 991752.893 1...",대덕구,목상동,5288


In [ ]:
car_hjd.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 차량 데이터/대전 차량 데이터.shp', encoding = 'utf-8')

QGIS 사용하여, 격자 중심 좌표 데이터와 결합

##### 행정동별 gid(격자) 개수 카운트

In [ ]:
## 작업 데이터 불러오기
car_hjd_point = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/car_hjd_point/car_hjd_point.shp', encoding = 'utf-8')
car_hjd_point.head(3)

,gid,area,population,BASE_DATE,ADM_DR_CD,ADM_DR_NM,구,행정동,계,geometry
0,다바962121,70838.62,NaN,20210630,2501053,효동,동구,효동,10515.0,POINT (996250.000 1812150.000)
1,다바959127,35781.60,NaN,20210630,2501053,효동,동구,효동,10515.0,POINT (995950.000 1812750.000)
2,다바954130,13042.94,278.0,20210630,2501053,효동,동구,효동,10515.0,POINT (995450.000 1813050.000)


In [ ]:
# 행정동 이름으로 그룹화하여 개수 카운트
a = car_hjd_point.groupby('ADM_DR_NM').count()

## 인덱스 리셋
a = a.reset_index()
a = a.reset_index(drop = True)

## 필요한 열만 추출
a = a[['ADM_DR_NM', 'gid']]
a

,ADM_DR_NM,gid
0,가수원동,989
1,가양1동,101
2,가양2동,178
3,가장동,56
4,갈마1동,148
...,...,...
74,판암1동,529
75,판암2동,78
76,홍도동,63
77,회덕동,1700


##### 동별 차량 등록 대수/행정동 격자 개수

In [ ]:
## 동별 차량 등록 대수와, 중심점 좌표 격자 데이터 결합
car_hjd_point = car_hjd_point.merge(a, on = 'ADM_DR_NM', how = 'left')
car_hjd_point.head(3)

,gid_x,area,population,BASE_DATE,ADM_DR_CD,ADM_DR_NM,구,행정동,계,geometry,gid_y
0,다바962121,70838.62,NaN,20210630,2501053,효동,동구,효동,10515.0,POINT (996250.000 1812150.000),242.0
1,다바959127,35781.60,NaN,20210630,2501053,효동,동구,효동,10515.0,POINT (995950.000 1812750.000),242.0
2,다바954130,13042.94,278.0,20210630,2501053,효동,동구,효동,10515.0,POINT (995450.000 1813050.000),242.0


In [ ]:
car_hjd_point['격자별차량현황'] = car_hjd_point['계']/car_hjd_point['gid_y']
car_hjd_point.head(3)

,gid_x,area,population,BASE_DATE,ADM_DR_CD,ADM_DR_NM,구,행정동,계,geometry,gid_y,격자별차량현황
0,다바962121,70838.62,NaN,20210630,2501053,효동,동구,효동,10515.0,POINT (996250.000 1812150.000),242.0,43.450413
1,다바959127,35781.60,NaN,20210630,2501053,효동,동구,효동,10515.0,POINT (995950.000 1812750.000),242.0,43.450413
2,다바954130,13042.94,278.0,20210630,2501053,효동,동구,효동,10515.0,POINT (995450.000 1813050.000),242.0,43.450413


In [ ]:
car_hjd_point.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 차량 데이터/대전시격자별차량현황.csv', encoding = 'utf-8', index = False)

### 모델링 데이터 만들기

#### 1. 수소충전소 버퍼별 값 대입하기

QGIS 사용하여 수소충전소 POINT 기준
각, 250m, 500m, 750m, 1000m 버퍼 생성하여
격자 데이터와 교차시킨 후, shp 파일로 저장

In [ ]:
## 250m 버퍼
buf_250 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/버퍼 교차/250 버퍼 교차.shp', encoding = 'cp949')
buf_250

,gid,area,population,이름,주소,위도,경도,geometry
0,다바932230,5501.03,NaN,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,"POLYGON ((993200.000 1823100.000, 993267.182 1..."
1,다바932232,1612.53,NaN,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,"POLYGON ((993200.000 1823300.000, 993289.936 1..."
2,다바966109,1363.66,82.0,대전중도가스수소충전소,대전광역시 동구 대전로 295,36.294452,127.460783,"POLYGON ((996600.000 1811000.000, 996618.210 1..."
3,다바931231,1052.15,9.0,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,"POLYGON ((993100.000 1823200.000, 993200.000 1..."
4,다바979080,902.54,NaN,낭월수소버스충전소,대전광역시 동구 금산로 455,36.267989,127.477952,"POLYGON ((997900.000 1808074.404, 997942.357 1..."
...,...,...,...,...,...,...,...,...
207,다바837155,NaN,NaN,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((983700.000 1815568.253, 983714.055 1..."
208,다바834154,NaN,NaN,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((983400.000 1815500.000, 983500.000 1..."
209,다바813179,NaN,NaN,대전유성수소충전소,대전광역시 유성구 현충원로 168,36.359694,127.290630,"POLYGON ((981300.000 1818000.000, 981400.000 1..."
210,다바856215,NaN,NaN,하이넷자운대충전소,대전광역시 유성구 자운로 62,36.393269,127.342414,"POLYGON ((985700.000 1821600.000, 985700.000 1..."


In [ ]:
## 500m 버퍼
buf_500 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/버퍼 교차/500 버퍼 교차.shp', encoding = 'cp949')
buf_500

,gid,area,population,이름,주소,위도,경도,geometry
0,다바838156,87955.33,0.0,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((983800.000 1815700.000, 983900.000 1..."
1,다바976081,13323.00,NaN,낭월수소버스충전소,대전광역시 동구 금산로 455,36.267989,127.477952,"POLYGON ((997600.000 1808122.474, 997615.103 1..."
2,다바933227,9653.28,286.0,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,"POLYGON ((993300.000 1822800.000, 993311.304 1..."
3,다바932236,7618.74,NaN,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,"POLYGON ((993200.000 1823698.948, 993206.893 1..."
4,다바932227,6343.77,667.0,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,"POLYGON ((993200.000 1822800.000, 993300.000 1..."
...,...,...,...,...,...,...,...,...
687,다바855220,NaN,NaN,하이넷자운대충전소,대전광역시 유성구 자운로 62,36.393269,127.342414,"POLYGON ((985500.000 1822096.086, 985503.914 1..."
688,다바835156,NaN,NaN,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((983500.000 1815700.000, 983600.000 1..."
689,다바854215,NaN,NaN,하이넷자운대충전소,대전광역시 유성구 자운로 62,36.393269,127.342414,"POLYGON ((985400.000 1821600.000, 985500.000 1..."
690,다바811175,NaN,NaN,대전유성수소충전소,대전광역시 유성구 현충원로 168,36.359694,127.290630,"POLYGON ((981100.000 1817600.000, 981200.000 1..."


In [ ]:
## 750m 버퍼
buf_750 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/버퍼 교차/750 버퍼 교차.shp', encoding = 'cp949')
buf_750

,gid,area,population,이름,주소,위도,경도,geometry
0,다바838156,87955.33,0.0,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((983800.000 1815700.000, 983900.000 1..."
1,다바976081,13323.00,NaN,낭월수소버스충전소,대전광역시 동구 금산로 455,36.267989,127.477952,"POLYGON ((997600.000 1808200.000, 997700.000 1..."
2,다바962115,11697.12,646.0,대전중도가스수소충전소,대전광역시 동구 대전로 295,36.294452,127.460783,"POLYGON ((996300.000 1811500.000, 996232.612 1..."
3,다바862211,10431.59,526.0,하이넷자운대충전소,대전광역시 유성구 자운로 62,36.393269,127.342414,"POLYGON ((986200.000 1821200.000, 986300.000 1..."
4,다바963115,10410.47,240.0,대전중도가스수소충전소,대전광역시 동구 대전로 295,36.294452,127.460783,"POLYGON ((996300.000 1811515.686, 996400.000 1..."
...,...,...,...,...,...,...,...,...
1424,다바831148,NaN,NaN,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((983100.000 1814900.000, 983200.000 1..."
1425,다바807186,NaN,NaN,대전유성수소충전소,대전광역시 유성구 현충원로 168,36.359694,127.290630,"POLYGON ((980800.000 1818600.000, 980718.125 1..."
1426,다바925235,NaN,NaN,신대수소버스충전소,대전광역시 대덕구 신탄진로315번길 72,36.406480,127.422514,"POLYGON ((992500.000 1823600.000, 992600.000 1..."
1427,다바919252,NaN,NaN,하이넷신탄진휴게소수소충전소,대전광역시 대덕구 산업단지로 27,36.428283,127.417995,"POLYGON ((992000.000 1825300.000, 992000.000 1..."


In [ ]:
## 1000m 버퍼
buf_1000 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/버퍼 교차/1000 버퍼 교차.shp', encoding = 'cp949')
buf_1000

,gid,area,population,이름,주소,위도,경도,geometry
0,다바838156,87955.33,0.0,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((983800.000 1815700.000, 983900.000 1..."
1,다바821176,14287.72,242.0,대전유성수소충전소,대전광역시 유성구 현충원로 168,36.359694,127.290630,"POLYGON ((982100.000 1817700.000, 982146.050 1..."
2,다바976081,13323.00,NaN,낭월수소버스충전소,대전광역시 동구 금산로 455,36.267989,127.477952,"POLYGON ((997600.000 1808200.000, 997700.000 1..."
3,다바962116,13176.56,687.0,대전중도가스수소충전소,대전광역시 동구 대전로 295,36.294452,127.460783,"POLYGON ((996200.000 1811700.000, 996300.000 1..."
4,다바962117,11799.37,598.0,대전중도가스수소충전소,대전광역시 동구 대전로 295,36.294452,127.460783,"POLYGON ((996200.000 1811749.848, 996300.000 1..."
...,...,...,...,...,...,...,...,...
2450,다바919252,NaN,NaN,하이넷신탄진휴게소수소충전소,대전광역시 대덕구 산업단지로 27,36.428283,127.417995,"POLYGON ((991900.000 1825300.000, 992000.000 1..."
2451,다바840152,NaN,NaN,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((984000.000 1815300.000, 984100.000 1..."
2452,다바842161,NaN,NaN,학하수소충전소,대전광역시 유성구 유성대로 437,36.335908,127.316275,"POLYGON ((984200.000 1816115.854, 984215.854 1..."
2453,다바859227,NaN,NaN,하이넷자운대충전소,대전광역시 유성구 자운로 62,36.393269,127.342414,"POLYGON ((985900.000 1822760.611, 986000.000 1..."


- 버퍼와 격자(그리드)가 교차되면 1, 교차되지 않으면 0 대입

###### 250m 버퍼

In [ ]:
a = []
for i in tqdm(range(len(total_gid))) :
  if total_gid.iloc[[i]].index in list(buf_250.index) :
    suso = 1
    a.append(suso)
  else : 
    suso = 0
    a.append(suso)

## 격자 데이터에 column 생성하여 추가
total_gid['buf_250'] = a
buf_250_gid = total_gid

## shp 파일로 저장
buf_250_gid.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 250.shp', encoding='utf-8')

100%|██████████| 54912/54912 [07:20<00:00, 124.74it/s]


###### 500m 버퍼

In [ ]:
a = []
for i in tqdm(range(len(total_gid))):
  if total_gid.iloc[[i]].index in list(buf_500.index):
    suso = 1
    a.append(suso)
  else : 
    suso = 0
    a.append(suso)

## 격자 데이터에 있는 250m 버퍼 column 삭제
total_gid.drop(['buf_250'], axis = 1, inplace = True)

## 격자 데이터에 column 생성하여 추가
total_gid['buf_500'] = a
buf_500_gid = total_gid

## shp 파일로 저장
buf_500_gid.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 500.shp', encoding='utf-8')

100%|██████████| 54912/54912 [20:31<00:00, 44.61it/s]


###### 750m 버퍼

In [ ]:
a = []
for i in tqdm(range(len(total_gid))) :
  if total_gid.iloc[[i]].index in list(buf_750.index) :
    suso = 1
    a.append(suso)
  else : 
    suso = 0
    a.append(suso)

## 격자 데이터에 있는 500m 버퍼 column 삭제
total_gid.drop(['buf_500'], axis = 1, inplace = True)

## 격자 데이터에 column 생성하여 추가
total_gid['buf_750'] = a
buf_750_gid = total_gid

## shp 파일로 저장
buf_750_gid.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 750.shp', encoding='utf-8')

 17%|█▋        | 9142/54912 [06:13<32:46, 23.28it/s]

###### 1000m 버퍼

In [ ]:
a = []
for i in tqdm(range(len(total_gid))):
  if total_gid.iloc[[i]].index in list(buf_1000.index):
    suso = 1
    a.append(suso)
  else : 
    suso = 0
    a.append(suso)

## 격자 데이터에 있는 750m 버퍼 column 삭제
total_gid.drop(['buf_750'], axis = 1, inplace = True)/

## 격자 데이터에 column 생성하여 추가
total_gid['buf_1000'] = a
buf_1000_gid = total_gid

## shp 파일로 저장
buf_1000_gid.to_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 1000.shp', encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


#### 최종 모델링 데이터 결합

- grid ID 기준으로 전처리한 데이터들 결합하여 모델링 할 때 사용할 최종 데이터셋 만들기

###### 데이터 불러오기

In [7]:
buffer_250 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 250.shp', encoding='utf-8')
buffer_250.head(3)

,gid,area,population,buf_250,geometry
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451..."
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424..."
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433..."


In [8]:
buffer_500 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 500.shp', encoding='utf-8')
buffer_500.head(3)

,gid,area,population,buf_500,geometry
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451..."
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424..."
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433..."


In [9]:
buffer_750 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 750.shp', encoding='utf-8')
buffer_750.head(3)

,gid,area,population,buf_750,geometry
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451..."
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424..."
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433..."


In [10]:
buffer_1000 = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/수소 버퍼 데이터/수소 버퍼 1000.shp', encoding='utf-8')
buffer_1000.drop(['business_n', 'school', 'school_new'], axis = 1, inplace = True)
buffer_1000.head(3)

,gid,area,population,buf_1000,geometry
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451..."
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424..."
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433..."


In [11]:
daejeon_school = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 학교데이터/대전 학교 가중치.csv')
daejeon_school = daejeon_school[['gid', 'school_new']]
daejeon_school.head(3)

,gid,school_new
0,다바922281,0.0
1,다바909251,0.0
2,다바905261,0.0


In [12]:
daejeon_link = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/도로 노드링크 데이터/도로 링크 데이터.shp')
daejeon_link = daejeon_link.rename(columns = {'cnt':'link_cnt'})
daejeon_link = daejeon_link[['gid', 'link_cnt']]
daejeon_link.head(3)

,gid,link_cnt
0,다바922281,0
1,다바909251,0
2,다바905261,2


In [13]:
daejeon_node = gpd.read_file('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/도로 노드링크 데이터/도로 노드 데이터.shp')
daejeon_node = daejeon_node.rename(columns = {'cnt':'node_cnt'})
daejeon_node = daejeon_node[['gid', 'node_cnt']]
daejeon_node.head(3)

,gid,node_cnt
0,다바922281,0
1,다바909251,0
2,다바905261,0


In [14]:
daejeon_fire = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 화재데이터/대전격자별화재건수.csv')
daejeon_fire = daejeon_fire[['gid_x', '격자별화재건수']]
daejeon_fire.head(3)

,gid_x,격자별화재건수
0,다바884169,0.289593
1,다바880166,0.289593
2,다바883165,0.289593


In [15]:
daejeon_business = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/대전 상권 가중치.csv')
daejeon_business = daejeon_business[['gid', 'business_new']]
daejeon_business.head(3)

,gid,business_new
0,다바922281,0.4
1,다바909251,1.0
2,다바905261,1.0


In [16]:
daejeon_business_2 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 상권데이터/대전 상권 분류.csv')
daejeon_business_2.drop(['Unnamed: 0'], axis = 1, inplace = True)
daejeon_business_2.head(3)

,gid,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육
0,다바922281,0,0,0,0,0,0,0,0
1,다바909251,0,0,2,78,1,1,31,1
2,다바905261,0,2,4,9,0,0,8,0


In [17]:
daejeon_car = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/전처리_후/대전 차량 데이터/대전시격자별차량현황.csv')
daejeon_car = daejeon_car[['gid_x', '격자별차량현황']]
daejeon_car.head(3)

,gid_x,격자별차량현황
0,다바962121,43.450413
1,다바959127,43.450413
2,다바954130,43.450413


###### 데이터 병합

In [18]:
## 250m 버퍼 데이터 합치기
H2_total_250 = pd.merge(buffer_250, daejeon_school, on = 'gid', how = 'left')
H2_total_250 = pd.merge(H2_total_250, daejeon_link, on = 'gid', how = 'left')
H2_total_250 = pd.merge(H2_total_250, daejeon_node, on = 'gid', how = 'left')
H2_total_250 = pd.merge(H2_total_250, daejeon_fire, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_250 = pd.merge(H2_total_250, daejeon_business, on = 'gid', how = 'left')
H2_total_250 = pd.merge(H2_total_250, daejeon_business_2, on = 'gid', how = 'left')
H2_total_250 = pd.merge(H2_total_250, daejeon_car, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_250.drop(['gid_x_x', 'gid_x_y'], axis = 1, inplace = True)
H2_total_250.head(3)

,gid,area,population,buf_250,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [19]:
## 500m 버퍼 데이터 합치기
H2_total_500 = pd.merge(buffer_500, daejeon_school, on = 'gid', how = 'left')
H2_total_500 = pd.merge(H2_total_500, daejeon_link, on = 'gid', how = 'left')
H2_total_500 = pd.merge(H2_total_500, daejeon_node, on = 'gid', how = 'left')
H2_total_500 = pd.merge(H2_total_500, daejeon_fire, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_500 = pd.merge(H2_total_500, daejeon_business, on = 'gid', how = 'left')
H2_total_500 = pd.merge(H2_total_500, daejeon_business_2, on = 'gid', how = 'left')
H2_total_500 = pd.merge(H2_total_500, daejeon_car, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_500.drop(['gid_x_x', 'gid_x_y'], axis = 1, inplace = True)
H2_total_500.head(3)

,gid,area,population,buf_500,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [20]:
## 750m 버퍼 데이터 합치기
H2_total_750 = pd.merge(buffer_750, daejeon_school, on = 'gid', how = 'left')
H2_total_750 = pd.merge(H2_total_750, daejeon_link, on = 'gid', how = 'left')
H2_total_750 = pd.merge(H2_total_750, daejeon_node, on = 'gid', how = 'left')
H2_total_750 = pd.merge(H2_total_750, daejeon_fire, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_750 = pd.merge(H2_total_750, daejeon_business, on = 'gid', how = 'left')
H2_total_750 = pd.merge(H2_total_750, daejeon_business_2, on = 'gid', how = 'left')
H2_total_750 = pd.merge(H2_total_750, daejeon_car, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_750.drop(['gid_x_x', 'gid_x_y'], axis = 1, inplace = True)
H2_total_750.head(3)

,gid,area,population,buf_750,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [21]:
## 1000m 버퍼 데이터 합치기
H2_total_1000 = pd.merge(buffer_1000, daejeon_school, on = 'gid', how = 'left')
H2_total_1000 = pd.merge(H2_total_1000, daejeon_link, on = 'gid', how = 'left')
H2_total_1000 = pd.merge(H2_total_1000, daejeon_node, on = 'gid', how = 'left')
H2_total_1000 = pd.merge(H2_total_1000, daejeon_fire, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_1000 = pd.merge(H2_total_1000, daejeon_business, on = 'gid', how = 'left')
H2_total_1000 = pd.merge(H2_total_1000, daejeon_business_2, on = 'gid', how = 'left')
H2_total_1000 = pd.merge(H2_total_1000, daejeon_car, left_on = 'gid', right_on = 'gid_x', how = 'left')
H2_total_1000.drop(['gid_x_x', 'gid_x_y'], axis = 1, inplace = True)
H2_total_1000.head(3)

,gid,area,population,buf_1000,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,다바922281,125401.97,NaN,1,"POLYGON ((127.41296 36.45045, 127.41296 36.451...",0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,다바909251,118444.81,NaN,1,"POLYGON ((127.39849 36.42339, 127.39849 36.424...",0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,다바905261,104399.92,11.0,1,"POLYGON ((127.39401 36.43240, 127.39401 36.433...",0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [22]:
## csv 파일로 저장하기
H2_total_250.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer250.csv', encoding = 'utf-8')
H2_total_500.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer500.csv', encoding = 'utf-8')
H2_total_750.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer750.csv', encoding = 'utf-8')
H2_total_1000.to_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer1000.csv', encoding = 'utf-8')

### 모델링

In [29]:
## 데이터 불러오기 (grid별 데이터 결합 완료한 데이터)
H2_total_250 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer250.csv')
H2_total_500 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer500.csv')
H2_total_750 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer750.csv')
H2_total_1000 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer1000.csv')

#### Linear SVC

##### 버퍼별 모델링 - 250m

###### 모델링 데이터 전처리

In [30]:
## gid번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_250.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [31]:
## null값 0으로 대체 
H2_total_250.fillna(0, inplace = True)

In [32]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_250 = H2_total_250.loc[(H2_total_250 != 0).any(axis = 1)]
H2_total_250.head(3)

,area,population,buf_250,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


###### 정규화 (StandardScaler)
평균 0, 분산 1인 정규분포로 치환

In [33]:
## 범주형변수이므로 정규화에서 제외시켜줌
H2_total_250_categorical = H2_total_250[['buf_250']]
H2_total_250_numeric = H2_total_250.drop(H2_total_250_categorical, axis=1)
H2_total_250_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [34]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
H2_total_250_numeric_scale = scaler.fit_transform(H2_total_250_numeric)

In [35]:
## 정규화 데이터 DF로 변환
H2_250_scale = pd.DataFrame(
    H2_total_250_numeric_scale,
    columns = list(H2_total_250_numeric.columns))
H2_250_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993


In [36]:
H2_250_scale.reset_index(drop = True, inplace = True)
H2_total_250_categorical.reset_index(drop = True, inplace = True)

In [37]:
## categorical 변수 추가
H2_250_model = pd.merge(H2_250_scale, H2_total_250_categorical,
         left_index = True, right_index = True, how = 'inner')
H2_250_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_250
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940,1
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993,1
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993,1


In [38]:
## target 변수 0과 1 비율 확인
H2_250_model['buf_250'].value_counts()/len(H2_250_model)

0    0.996077
1    0.003923
Name: buf_250, dtype: float64

###### Train/Test 데이터셋 분리

In [39]:
from sklearn.model_selection import train_test_split

x_250 = H2_250_model.drop(['buf_250'], axis = 1)
y_250 = H2_250_model['buf_250']

###### Modeling - Linear SVC

In [40]:
from sklearn.svm import SVC, SVR

In [41]:
clf = SVC(C = 0.1, kernel = 'linear', random_state = 42)
clf.fit(x_250, y_250)

print('가중치:', clf.coef_)
print('절편항:', clf.intercept_)
print('정확도:', np.mean(y_250 == clf.predict(x_250)))

가중치: [[ 2.10399907e+00 -3.32806098e-02  3.25860960e-02 -1.03766579e-01
   5.67488194e-02  3.81494371e-02  1.60675450e-02  2.38353144e-02
   2.47349653e-03 -7.23305552e-02  8.82059768e-02  8.67361738e-19
   1.33755142e-02  5.19909340e-02  2.88854738e-03 -2.09404196e-02]]
절편항: [-11.5835961]
정확도: 0.9999074656697635


In [42]:
pd.DataFrame(clf.coef_,columns = x_250.columns)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,2.103999,-0.033281,0.032586,-0.103767,0.056749,0.038149,0.016068,0.023835,0.002473,-0.072331,0.088206,8.673617e-19,0.013376,0.051991,0.002889,-0.02094


##### 버퍼별 모델링 - 500m

###### 모델링 데이터 전처리

In [43]:
## gid번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_500.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [44]:
## null값 0으로 대체 
H2_total_500.fillna(0, inplace = True)

In [45]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_500 = H2_total_500.loc[(H2_total_500 != 0).any(axis = 1)]
H2_total_500.head(3)

,area,population,buf_500,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


###### 정규화 (StandardScaler)
평균 0, 분산 1인 정규분포로 치환

In [46]:
## 범주형변수이므로 정규화에서 제외시켜줌
H2_total_500_categorical = H2_total_500[['buf_500']]
H2_total_500_numeric = H2_total_500.drop(H2_total_500_categorical, axis=1)
H2_total_500_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
H2_total_500_numeric_scale = scaler.fit_transform(H2_total_500_numeric)

In [48]:
## 정규화 데이터 DF로 변환
H2_500_scale = pd.DataFrame(
    H2_total_500_numeric_scale,
    columns = list(H2_total_500_numeric.columns))
H2_500_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993


In [49]:
H2_500_scale.reset_index(drop = True, inplace = True)
H2_total_500_categorical.reset_index(drop = True, inplace = True)

In [50]:
## categorical 변수 추가
H2_500_model = pd.merge(H2_500_scale, H2_total_500_categorical,
         left_index = True, right_index = True, how = 'inner')
H2_500_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_500
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940,1
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993,1
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993,1


In [51]:
## target 변수 0과 1 비율 확인
H2_500_model['buf_500'].value_counts()/len(H2_500_model)

0    0.987193
1    0.012807
Name: buf_500, dtype: float64

###### Train/Test 데이터셋 분리

In [52]:
from sklearn.model_selection import train_test_split

x_500 = H2_500_model.drop(['buf_500'], axis = 1)
y_500 = H2_500_model['buf_500']

###### Modeling - Linear SVC

In [53]:
from sklearn.svm import SVC, SVR

In [54]:
clf = SVC(C = 0.1, kernel = 'linear', random_state = 42)
clf.fit(x_500, y_500)

print('가중치:', clf.coef_)
print('절편항:', clf.intercept_)
print('정확도:', np.mean(y_500 == clf.predict(x_500)))

가중치: [[ 3.52642660e+00 -4.34238018e-02 -1.96156100e-02 -1.99055699e-02
   7.95097067e-03  1.17607447e-02  2.55061213e-02 -4.45769082e-02
  -4.60804681e-02  1.15706875e-01 -1.96539679e-02 -1.73477926e-01
  -5.82716514e-02 -1.25719544e-02  1.68483881e-02 -3.40544442e-04]]
절편항: [-12.52775949]
정확도: 0.9995928489469593


In [55]:
pd.DataFrame(clf.coef_,columns = x_500.columns)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,3.526427,-0.043424,-0.019616,-0.019906,0.007951,0.011761,0.025506,-0.044577,-0.04608,0.115707,-0.019654,-0.173478,-0.058272,-0.012572,0.016848,-0.000341


##### 버퍼별 모델링 - 750m

###### 모델링 데이터 전처리

In [56]:
## gid번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_750.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [57]:
## null값 0으로 대체 
H2_total_750.fillna(0, inplace = True)

In [58]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_750 = H2_total_750.loc[(H2_total_750 != 0).any(axis = 1)]
H2_total_750.head(3)

,area,population,buf_750,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


###### 정규화 (StandardScaler)
평균 0, 분산 1인 정규분포로 치환

In [59]:
## 범주형변수이므로 정규화에서 제외시켜줌
H2_total_750_categorical = H2_total_750[['buf_750']]
H2_total_750_numeric = H2_total_750.drop(H2_total_750_categorical, axis=1)
H2_total_750_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [60]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
H2_total_750_numeric_scale = scaler.fit_transform(H2_total_750_numeric)

In [61]:
## 정규화 데이터 DF로 변환
H2_750_scale = pd.DataFrame(
    H2_total_750_numeric_scale,
    columns = list(H2_total_750_numeric.columns))
H2_750_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993


In [62]:
H2_750_scale.reset_index(drop = True, inplace = True)
H2_total_750_categorical.reset_index(drop = True, inplace = True)

In [63]:
## categorical 변수 추가
H2_750_model = pd.merge(H2_750_scale, H2_total_750_categorical,
         left_index = True, right_index = True, how = 'inner')
H2_750_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_750
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940,1
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993,1
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993,1


In [64]:
## target 변수 0과 1 비율 확인
H2_750_model['buf_750'].value_counts()/len(H2_750_model)

0    0.973554
1    0.026446
Name: buf_750, dtype: float64

###### Train/Test 데이터셋 분리

In [65]:
from sklearn.model_selection import train_test_split

x_750 = H2_750_model.drop(['buf_750'], axis = 1)
y_750 = H2_750_model['buf_750']

###### Modeling - Linear SVC

In [66]:
from sklearn.svm import SVC, SVR

In [67]:
clf = SVC(C = 0.1, kernel = 'linear', random_state = 42)
clf.fit(x_750, y_750)

print('가중치:', clf.coef_)
print('절편항:', clf.intercept_)
print('정확도:', np.mean(y_750 == clf.predict(x_750)))

가중치: [[ 3.97519679e+00 -7.46115094e-03  4.02798348e-02  1.09890947e-01
  -1.58723014e-01 -8.29188422e-02  6.36611074e-02 -2.52123584e-02
  -1.20198582e-02  5.15463494e-02 -8.62689952e-02  1.12257614e-02
  -2.18883055e-02 -3.08703401e-02  6.32832711e-04  5.56022191e-02]]
절편항: [-8.54788451]
정확도: 0.9996483695451013


In [68]:
pd.DataFrame(clf.coef_,columns = x_750.columns)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,3.975197,-0.007461,0.04028,0.109891,-0.158723,-0.082919,0.063661,-0.025212,-0.01202,0.051546,-0.086269,0.011226,-0.021888,-0.03087,0.000633,0.055602


##### 버퍼별 모델링 - 1000m

###### 모델링 데이터 전처리

In [69]:
## gid번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_1000.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [70]:
## null값 0으로 대체 
H2_total_1000.fillna(0, inplace = True)

In [71]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_1000 = H2_total_1000.loc[(H2_total_1000 != 0).any(axis = 1)]
H2_total_1000.head(3)

,area,population,buf_1000,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


###### 정규화 (StandardScaler)
평균 0, 분산 1인 정규분포로 치환

In [72]:
## 범주형변수이므로 정규화에서 제외시켜줌
H2_total_1000_categorical = H2_total_1000[['buf_1000']]
H2_total_1000_numeric = H2_total_1000.drop(H2_total_1000_categorical, axis=1)
H2_total_1000_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [73]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
H2_total_1000_numeric_scale = scaler.fit_transform(H2_total_1000_numeric)

In [74]:
## 정규화 데이터 DF로 변환
H2_1000_scale = pd.DataFrame(
    H2_total_1000_numeric_scale,
    columns = list(H2_total_1000_numeric.columns))
H2_1000_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993


In [75]:
H2_1000_scale.reset_index(drop = True, inplace = True)
H2_total_1000_categorical.reset_index(drop = True, inplace = True)

In [76]:
## categorical 변수 추가
H2_1000_model = pd.merge(H2_1000_scale, H2_total_1000_categorical,
         left_index = True, right_index = True, how = 'inner')
H2_1000_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_1000
0,48.064529,-0.260191,-0.310515,-0.432726,-0.224632,0.972570,0.255371,-0.113208,-0.147928,-0.224408,-0.133883,-0.060045,-0.093301,-0.199126,-0.126492,-0.167940,1
1,45.387606,-0.260191,-0.310515,-0.432726,-0.224632,-0.135186,1.778129,-0.113208,-0.147928,1.504248,22.806723,5.486070,5.810032,14.109449,1.192220,0.546993,1
2,39.983521,-0.150794,-0.310515,0.673456,-0.224632,1.445134,1.778129,-0.113208,5.757608,3.232903,2.513110,-0.060045,-0.093301,3.493409,-0.126492,0.546993,1


In [77]:
## target 변수 0과 1 비율 확인
H2_1000_model['buf_1000'].value_counts()/len(H2_1000_model)

0    0.954566
1    0.045434
Name: buf_1000, dtype: float64

###### Train/Test 데이터셋 분리

In [78]:
from sklearn.model_selection import train_test_split

x_1000 = H2_1000_model.drop(['buf_1000'], axis = 1)
y_1000 = H2_1000_model['buf_1000']

###### Modeling - Linear SVC

In [79]:
from sklearn.svm import SVC, SVR

In [80]:
clf = SVC(C = 0.1, kernel = 'linear', random_state = 42)
clf.fit(x_1000, y_1000)

print('가중치:', clf.coef_)
print('절편항:', clf.intercept_)
print('정확도:', np.mean(y_1000 == clf.predict(x_1000)))

가중치: [[ 4.97401545e+00  7.08311891e-02 -5.34750032e-02 -6.21529746e-02
   6.13082350e-02 -1.21320780e-02 -6.70424646e-02  2.18251674e-02
   9.03930773e-04  2.82723405e-03 -3.10588321e-03 -2.56658019e-02
  -5.07121562e-03 -5.26698835e-03  3.67255472e-03  5.55367424e-02]]
절편항: [-4.0529826]
정확도: 0.9993152459562498


In [81]:
pd.DataFrame(clf.coef_,columns = x_1000.columns)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,4.974015,0.070831,-0.053475,-0.062153,0.061308,-0.012132,-0.067042,0.021825,0.000904,0.002827,-0.003106,-0.025666,-0.005071,-0.005267,0.003673,0.055537


#### Pycaret

In [5]:
## 데이터 불러오기 (grid별 데이터 결합 완료한 데이터)
H2_total_250 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer250.csv')
H2_total_500 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer500.csv')
H2_total_750 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer750.csv')
H2_total_1000 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer1000.csv')

##### 버퍼별 모델링 - 250m

- 수소 충전소가 버퍼 250m 기준으로 1(수소충전소 있다)을 넣은 경우

###### 모델링 데이터 전처리

In [6]:
## gid 번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_250.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [7]:
## null값 0으로 대체
H2_total_250.fillna(0, inplace = True)
H2_total_250.isnull().sum()

area            0
population      0
buf_250         0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [8]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_250 = H2_total_250.loc[(H2_total_250 != 0).any(axis = 1)]
H2_total_250 # 54912 -> 54034행으로 변경

,area,population,buf_250,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,102749.66,0.0,1,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,95862.58,0.0,1,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,0.00,0.0,0,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,0.00,0.0,0,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,0.00,0.0,0,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,0.00,0.0,0,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [9]:
## target 값 (수소충전소 여부) 분포 확인
H2_total_250['buf_250'].value_counts()

0    53822
1      212
Name: buf_250, dtype: int64

In [10]:
## 정규화 진행하기 위하여 target 변수 제외
H2_total_250_target = H2_total_250[['buf_250']]
H2_total_250_numeric = H2_total_250.drop(columns = 'buf_250', axis = 1)
H2_total_250_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [11]:
## MinMaxScaler스케일링을 통한 데이터 정규화
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
H2_total_250_numeric_scale = scaler.fit_transform(H2_total_250_numeric)

In [12]:
## 정규화 데이터 DF(dataframe)로 변환
H2_250_scale = pd.DataFrame(
    H2_total_250_numeric_scale,
    columns = list(H2_total_250_numeric.columns))
H2_250_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737


In [13]:
## categorical 변수 추가
H2_250_model = pd.merge(H2_250_scale, H2_total_250_target,
         left_index = True, right_index = True, how = 'inner')
H2_250_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_250
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514,1
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737,1
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737,1


##### Train/Test 데이터셋 분류

In [14]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(H2_250_model, test_size = 0.2, stratify = H2_250_model['buf_250'])

In [15]:
## train셋의 target 변수 0과 1 비율 보기
train['buf_250'].value_counts()/len(train)

0    0.996004
1    0.003996
Name: buf_250, dtype: float64

In [16]:
## test셋의 target 변수 0과 1 비율 보기
test['buf_250'].value_counts()/len(test)

0    0.996051
1    0.003949
Name: buf_250, dtype: float64

target 값의 비율이 고르지 못하여 오버샘플링(oversampling) 진행

###### Test 데이터셋 오버샘플링

In [17]:
## 필요 library load
from imblearn.over_sampling import RandomOverSampler,SMOTE,BorderlineSMOTE, ADASYN
from collections import Counter

In [18]:
## 오버샘플링 적용을 위한 x, y값 분류
x_train = train.drop(columns = ['buf_250'])
y_train = train[['buf_250']]

In [19]:
## y값 오버샘플링
smote = SMOTE(random_state = 42, k_neighbors = 5)
X_smote,y_smote = smote.fit_resample(x_train, y_train)

print('ddd %s'%Counter(y_train))
print('ddd %s'%Counter(y_smote))

ddd Counter({'buf_250': 1})
ddd Counter({'buf_250': 1})


In [20]:
## 오버샘플링한 x,y다시 결합
train = pd.merge(X_smote, y_smote, left_index = True, right_index = True)

In [21]:
## y값의 비율 재확인 - y 값의 비율이 일정하게 잘 샘플링 된 것을 확인 가능
train[['buf_250']].value_counts()/len(train)

buf_250
0          0.5
1          0.5
dtype: float64

###### pycaret 사용하여 모델 선택

In [22]:
## 모델 선택
reg = setup(train, target = 'buf_250')

,Description,Value
0,session_id,8649
1,Target,buf_250
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(84740, 17)"
5,Missing Values,False
6,Numeric Features,16
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='buf_250',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_stra...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [23]:
top3 = compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.128
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.631
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.079
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,4.647
et,Extra Trees Classifier,0.9999,1.0000,1.0000,0.9998,0.9999,0.9998,0.9998,1.972
lightgbm,Light Gradient Boosting Machine,0.9999,1.0000,1.0000,0.9999,0.9999,0.9998,0.9998,0.537
knn,K Neighbors Classifier,0.9973,0.9990,1.0000,0.9946,0.9973,0.9946,0.9946,2.616
lr,Logistic Regression,0.9945,1.0000,1.0000,0.9891,0.9945,0.9890,0.9891,0.596
svm,SVM - Linear Kernel,0.9940,0.0000,1.0000,0.9882,0.9940,0.9880,0.9881,0.131
lda,Linear Discriminant Analysis,0.9667,0.9930,0.9681,0.9653,0.9667,0.9334,0.9334,0.162


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=8649, splitter='best'), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1,

In [24]:
## 하이퍼파라미터 튜닝
tuned_top3 =  [tune_model(i) for i in top3]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000
4,0.9998,0.9998,1.0,0.9997,0.9998,0.9997,0.9997
5,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0,1.0000,1.0000,1.0000,1.0000


INFO:logs:create_model_container: 17
INFO:logs:master_model_container: 17
INFO:logs:display_container: 5
INFO:logs:AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                   learning_rate=0.005, n_estimators=140, random_state=8649)
INFO:logs:tune_model() succesfully completed......................................


In [25]:
## 블랜딩해서 Voting 방식으로 앙상블
blend_top3 = blend_models(estimator_list = tuned_top3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000
4,0.9998,1.0,1.0,0.9997,0.9998,0.9997,0.9997
5,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0,1.0,1.0000,1.0000,1.0000,1.0000


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 6
INFO:logs:VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=5,
                                                     max_features='sqrt',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.1,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=5,
                                                     min_weight_fraction_leaf=0.0,
                                 

In [26]:
## 최종 모델
final_model_250 = finalize_model(blend_top3) 

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=5,
                                                     max_features='sqrt',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.1,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=5,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated'

In [27]:
## k-fold를 통해 모델의 성능 측정을 위해서 X, Y값 분리해주기
train_Y = train[['buf_250']]
train_X = train.drop(columns = ['buf_250'])

In [28]:
## train_set을 집어넣어서 k-겹 교차 검증 하기
from sklearn.model_selection import cross_val_score

scores = cross_val_score(final_model_250, # 학습한 모델
                       train_X, # X
                       train_Y, # y
                       cv=6) 
scores

array([1.        , 1.        , 1.        , 0.99992919, 1.        ,
       1.        ])

In [29]:
## 테스트 데이터셋으로 모델 성능 최종 확인
# x,y값 분리
test_Y = test[['buf_250']]
test_X = test.drop(columns = ['buf_250'])

In [30]:
## 인덱스 리셋
test_X.reset_index(inplace = True, drop = True)
test_Y.reset_index(inplace = True, drop = True)

In [31]:
## test셋에 예측 모델 적용
prediction = predict_model(final_model_250, data = test_X)
prediction.reset_index(inplace = True)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=5,
                                                     max_features='sqrt',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.1,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=5,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',


In [32]:
## 값이 일치하지 않는 행이 몇개 있는지 확인
prediction[prediction['Label'] != test_Y['buf_250']]

,index,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,Label,Score


- 수소 충전소별 250m 버퍼에 해당하는 격자를 1로 나타낸 데이터 모델 : 추후 LPG 250m 버퍼에 해당하는 격자들에 적용할 모델

##### 버퍼별 모델링 - 500m

- 수소 충전소가 버퍼 500m 기준으로 1(수소충전소 있다)을 넣은 경우

###### 모델링 데이터 전처리

In [33]:
## gid 번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_500.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [34]:
## null값 0으로 대체
H2_total_500.fillna(0, inplace = True)
H2_total_500.isnull().sum()

area            0
population      0
buf_500         0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [35]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_500 = H2_total_500.loc[(H2_total_500 != 0).any(axis = 1)]
H2_total_500 # 54912 -> 54034행으로 변경

,area,population,buf_500,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,102749.66,0.0,1,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,95862.58,0.0,1,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,0.00,0.0,0,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,0.00,0.0,0,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,0.00,0.0,0,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,0.00,0.0,0,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [36]:
## target 값 (수소충전소 여부) 분포 확인
H2_total_500['buf_500'].value_counts()

0    53342
1      692
Name: buf_500, dtype: int64

In [37]:
## 정규화 진행하기 위하여 target 변수 제외
H2_total_500_target = H2_total_500[['buf_500']]
H2_total_500_numeric = H2_total_500.drop(columns = 'buf_500', axis = 1)
H2_total_500_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [38]:
## MinMaxScaler스케일링을 통한 데이터 정규화
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
H2_total_500_numeric_scale = scaler.fit_transform(H2_total_500_numeric)

In [39]:
# 정규화 데이터 DF(dataframe)로 변환
H2_500_scale = pd.DataFrame(
    H2_total_500_numeric_scale,
    columns = list(H2_total_500_numeric.columns))
H2_500_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737


In [40]:
# categorical 변수 추가
H2_500_model = pd.merge(H2_500_scale, H2_total_500_target,
         left_index = True, right_index = True, how = 'inner')
H2_500_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_500
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514,1
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737,1
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737,1


###### Train/Test 데이터셋 분류

In [41]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(H2_500_model, test_size = 0.2, stratify = H2_500_model['buf_500'])

In [42]:
## train셋의 target 변수 0과 1 비율 보기
train['buf_500'].value_counts()/len(train)

0    0.986977
1    0.013023
Name: buf_500, dtype: float64

target 값의 비율이 고르지 못하여 오버샘플링(oversampling) 진행

###### Test 데이터셋 오버샘플링

In [43]:
## 필요 library load
from imblearn.over_sampling import RandomOverSampler,SMOTE,BorderlineSMOTE, ADASYN
from collections import Counter

In [44]:
## 오버샘플링 적용을 위한 x,y값 분류
x_train = train.drop(columns = ['buf_500'])
y_train = train[['buf_500']]

In [45]:
## y값 오버샘플링
smote = SMOTE(random_state = 42, k_neighbors = 5)
X_smote,y_smote = smote.fit_resample(x_train,y_train)

print('ddd %s'%Counter(y_train))
print('ddd %s'%Counter(y_smote))

ddd Counter({'buf_500': 1})
ddd Counter({'buf_500': 1})


In [46]:
## 오버샘플링한 x,y다시 결합
train=pd.merge(X_smote,y_smote, left_index = True, right_index = True)

In [47]:
## y값의 비율 재확인
train[['buf_500']].value_counts()/len(train)

buf_500
0          0.5
1          0.5
dtype: float64

###### pycaret 사용하여 모델 선택

In [48]:
## 모델 선택
reg = setup(train, target = 'buf_500')

,Description,Value
0,session_id,560
1,Target,buf_500
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(83972, 17)"
5,Missing Values,False
6,Numeric Features,16
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='buf_500',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_stra...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [49]:
top3 = compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.068
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.202
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.075
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,4.433
lightgbm,Light Gradient Boosting Machine,0.9999,1.0000,0.9999,0.9999,0.9999,0.9998,0.9998,0.454
et,Extra Trees Classifier,0.9991,1.0000,1.0000,0.9983,0.9992,0.9983,0.9983,2.128
lr,Logistic Regression,0.9923,0.9993,1.0000,0.9848,0.9924,0.9846,0.9847,0.325
svm,SVM - Linear Kernel,0.9923,0.0000,1.0000,0.9848,0.9924,0.9846,0.9847,0.076
knn,K Neighbors Classifier,0.9907,0.9963,1.0000,0.9818,0.9908,0.9814,0.9815,2.397
lda,Linear Discriminant Analysis,0.9643,0.9910,0.9650,0.9639,0.9644,0.9286,0.9287,0.161


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=560, splitter='best'), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, 

In [51]:
## 하이퍼파라미터 튜닝
tuned_top3 =  [tune_model(i) for i in top3]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 6
INFO:logs:AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=0.0005,
                   n_estimators=110, random_state=560)
INFO:logs:tune_model() succesfully completed......................................


In [52]:
## 블랜딩해서 Voting 방식으로 앙상블
blend_top3 = blend_models(estimator_list=tuned_top3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0


INFO:logs:create_model_container: 19
INFO:logs:master_model_container: 19
INFO:logs:display_container: 7
INFO:logs:VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=15,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.005,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                   

In [53]:
## 최종 모델 선택
final_model_500 = finalize_model(blend_top3)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=15,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.005,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',


In [54]:
## k-fold를 통해 모델의 성능 측정을 위해서 X, Y값 분리해주기
train_Y = train[['buf_500']]
train_X = train.drop(columns = ['buf_500'])

In [55]:
## train_set을 집어넣어서 k-겹 교차 검증 하기
from sklearn.model_selection import cross_val_score

scores=cross_val_score(final_model_500, # 학습한 모델
                       train_X, # X
                       train_Y, # y
                       cv = 6) 
scores

array([1., 1., 1., 1., 1., 1.])

In [56]:
## 테스트 데이터셋으로 모델 성능 최종 확인
# x,y값 분리
test_Y = test[['buf_500']]
test_X = test.drop(columns = ['buf_500'])

In [57]:
## 인덱스 리셋
test_X.reset_index(inplace = True, drop = True)
test_Y.reset_index(inplace = True, drop = True)

In [58]:
## test셋에 예측 모델 적용
prediction = predict_model(final_model_500, data = test_X)
prediction.reset_index(inplace = True)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=15,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.005,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
  

In [59]:
## 값이 일치하지 않는 행이 몇개 있는지 확인
prediction[prediction['Label'] != test_Y['buf_500']]

,index,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,Label,Score


- 수소 충전소별 500m 버퍼에 해당하는 격자를 1로 나타낸 데이터 모델 : 추후 LPG 500m 버퍼에 해당하는 격자들에 적용할 모델

##### 버퍼별 모델링 - 750m

- 수소 충전소가 버퍼 750m 기준으로 1(수소충전소 있다)을 넣은 경우

###### 모델링 데이터 전처리

In [60]:
## gid번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_750.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [61]:
## null값 0으로 대체
H2_total_750.fillna(0, inplace = True)
H2_total_750.isnull().sum()

area            0
population      0
buf_750         0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [62]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_750 = H2_total_750.loc[(H2_total_750 != 0).any(axis = 1)]
H2_total_750 # 54912 -> 54034행으로 변경

,area,population,buf_750,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,102749.66,0.0,1,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,95862.58,0.0,1,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,0.00,0.0,0,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,0.00,0.0,0,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,0.00,0.0,0,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,0.00,0.0,0,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [63]:
## target 값 (수소충전소 여부) 분포 확인
H2_total_750['buf_750'].value_counts()

0    52605
1     1429
Name: buf_750, dtype: int64

In [64]:
## 정규화 진행하기 위하여 target 변수 제외
H2_total_750_target = H2_total_750[['buf_750']]
H2_total_750_numeric = H2_total_750.drop(columns = 'buf_750', axis = 1)
H2_total_750_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [65]:
## MinMaxScaler스케일링을 통한 데이터 정규화
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
H2_total_750_numeric_scale = scaler.fit_transform(H2_total_750_numeric)

In [66]:
## 정규화 데이터 DF(dataframe)로 변환
H2_750_scale = pd.DataFrame(
    H2_total_750_numeric_scale,
    columns = list(H2_total_750_numeric.columns))
H2_750_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737


In [67]:
## categorical 변수 추가
H2_750_model = pd.merge(H2_750_scale, H2_total_750_target,
         left_index = True, right_index = True, how = 'inner')
H2_750_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_750
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514,1
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737,1
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737,1


###### Train/Test 데이터셋 분류

In [68]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(H2_750_model, test_size = 0.2, stratify = H2_750_model['buf_750'])

In [69]:
## train셋의 target 변수 0과 1 비율 보기
train['buf_750'].value_counts()/len(train)

0    0.973131
1    0.026869
Name: buf_750, dtype: float64

In [70]:
## test셋의 target 변수 0과 1 비율 보기
test['buf_750'].value_counts()/len(test)

0    0.973108
1    0.026892
Name: buf_750, dtype: float64

target 값의 비율이 고르지 못하여 오버샘플링(oversampling) 진행

In [71]:
## 오버샘플링 적용을 위한 x,y값 분류
x_train = train.drop(columns = ['buf_750'])
y_train = train[['buf_750']]

In [72]:
## y값 오버샘플링
smote = SMOTE(random_state = 42, k_neighbors = 5)
X_smote,y_smote = smote.fit_resample(x_train,y_train)

print('ddd %s'%Counter(y_train))
print('ddd %s'%Counter(y_smote))

ddd Counter({'buf_750': 1})
ddd Counter({'buf_750': 1})


In [73]:
## 오버샘플링한 x,y다시 결합
train = pd.merge(X_smote, y_smote,left_index = True, right_index = True)

In [74]:
## y값의 비율 재확인
train[['buf_750']].value_counts()/len(train)

buf_750
0          0.5
1          0.5
dtype: float64

###### pycaret 사용하여 모델 선택

In [75]:
## 모델 선택
reg = setup(train, target = 'buf_750')

,Description,Value
0,session_id,2556
1,Target,buf_750
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(82794, 17)"
5,Missing Values,False
6,Numeric Features,16
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='buf_750',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_stra...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [76]:
top3 = compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.068
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.292
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.076
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,4.299
lightgbm,Light Gradient Boosting Machine,0.9998,1.0000,0.9998,0.9998,0.9998,0.9996,0.9996,0.520
et,Extra Trees Classifier,0.9987,1.0000,1.0000,0.9975,0.9987,0.9975,0.9975,2.191
svm,SVM - Linear Kernel,0.9933,0.0000,1.0000,0.9868,0.9933,0.9866,0.9867,0.079
lr,Logistic Regression,0.9902,0.9993,0.9940,0.9865,0.9902,0.9804,0.9804,0.511
knn,K Neighbors Classifier,0.9859,0.9952,0.9998,0.9728,0.9861,0.9718,0.9722,1.943
qda,Quadratic Discriminant Analysis,0.9584,0.9906,1.0000,0.9233,0.9601,0.9168,0.9200,0.074


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=2556, splitter='best'), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1,

In [77]:
## 하이퍼파라미터 튜닝
tuned_top3 =  [tune_model(i) for i in top3]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0


INFO:logs:create_model_container: 17
INFO:logs:master_model_container: 17
INFO:logs:display_container: 5
INFO:logs:AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.1,
                   n_estimators=140, random_state=2556)
INFO:logs:tune_model() succesfully completed......................................


In [78]:
## 블랜딩해서 보팅방식으로 앙상블
blend_top3 = blend_models(estimator_list = tuned_top3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 6
INFO:logs:VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=9,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.3,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                   

In [79]:
## 최종 모델
final_model_750 = finalize_model(blend_top3)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=9,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.3,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',


In [80]:
## k-fold를 통해 모델의 성능 측정을 위해서 X,Y값 분리해주기
train_Y = train[['buf_750']]
train_X = train.drop(columns=['buf_750'])

In [81]:
## train_set을 집어넣어서 k-겹 교차 검증 하기
from sklearn.model_selection import cross_val_score

scores=cross_val_score(final_model_750, # 학습한 모델
                       train_X, # X
                       train_Y, # y
                       cv = 6) 
scores

array([1.        , 1.        , 1.        , 1.        , 0.99992753,
       1.        ])

In [82]:
## 테스트 데이터셋으로 모델 성능 최종 확인
## x,y값 분리
test_Y = test[['buf_750']]
test_X = test.drop(columns = ['buf_750'])

In [83]:
## 인덱스 리셋
test_X.reset_index(inplace = True, drop = True)
test_Y.reset_index(inplace = True, drop = True)

In [84]:
## test셋에 예측 모델 적용
prediction = predict_model(final_model_750, data = test_X)
prediction.reset_index(inplace = True)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=9,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.3,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
  

In [85]:
## 값이 일치하지 않는 행이 몇개 있는지 확인
prediction[prediction['Label'] != test_Y['buf_750']]

,index,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,Label,Score


##### 버퍼별 모델링 - 1000m

###### 모델링 데이터 전처리

In [86]:
## gid 번호와 geometry 열 제거 (모델링에 필요하지 않음)
H2_total_1000.drop(['gid', 'geometry', 'Unnamed: 0'], axis = 1, inplace = True)

In [87]:
## null값 0으로 대체
H2_total_1000.fillna(0, inplace = True)
H2_total_1000.isnull().sum()

area            0
population      0
buf_1000        0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [88]:
## 전체 열이 0인 값은 모델링에 영향을 끼치지 않는 데이터라고 판단하여 제거
H2_total_1000 = H2_total_1000.loc[(H2_total_1000 != 0).any(axis = 1)]
H2_total_1000 # 54912 -> 54034행으로 변경

,area,population,buf_1000,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,1,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,1,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,1,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,102749.66,0.0,1,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,95862.58,0.0,1,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,0.00,0.0,0,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,0.00,0.0,0,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,0.00,0.0,0,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,0.00,0.0,0,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [89]:
## target 값 (수소충전소 여부) 분포 확인
H2_total_1000['buf_1000'].value_counts()

0    51579
1     2455
Name: buf_1000, dtype: int64

In [90]:
## 정규화 진행하기 위하여 target 변수 제외
H2_total_1000_target = H2_total_1000[['buf_1000']]
H2_total_1000_numeric = H2_total_1000.drop(columns = 'buf_1000', axis = 1)
H2_total_1000_numeric.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,118444.81,0.0,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,104399.92,11.0,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468


In [91]:
## MinMaxScaler스케일링을 통한 데이터 정규화
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
H2_total_1000_numeric_scale = scaler.fit_transform(H2_total_1000_numeric)

In [92]:
## 정규화 데이터 DF(dataframe)로 변환
H2_1000_scale = pd.DataFrame(
    H2_total_1000_numeric_scale,
    columns = list(H2_total_1000_numeric.columns))
H2_1000_scale.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737


In [93]:
## categorical 변수 추가
H2_1000_model = pd.merge(H2_1000_scale, H2_total_1000_target,
         left_index = True, right_index = True, how = 'inner')
H2_1000_model.head(3)

,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,buf_1000
0,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.056514,1
1,0.944521,0.000000,0.0,0.000000,0.0,0.041610,1.0,0.0,0.000000,0.032787,0.208,0.111111,0.142857,0.484375,0.019608,0.146737,1
2,0.832522,0.002131,0.0,0.090909,0.0,0.189394,1.0,0.0,0.181818,0.065574,0.024,0.000000,0.000000,0.125000,0.000000,0.146737,1


###### Train/Test 데이터셋 분류

In [94]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(H2_1000_model, test_size = 0.2, stratify = H2_1000_model['buf_1000'])

In [95]:
## train셋의 target 변수 0과 1 비율 보기
train['buf_1000'].value_counts()/len(train)

0    0.953832
1    0.046168
Name: buf_1000, dtype: float64

In [96]:
## test셋의 target 변수 0과 1 비율 보기
test['buf_1000'].value_counts()/len(test)

0    0.953832
1    0.046168
Name: buf_1000, dtype: float64

target 값의 비율이 고르지 못하여 오버샘플링(oversampling) 진행

###### Test 데이터셋 오버샘플링

In [97]:
## 필요 library load
from imblearn.over_sampling import RandomOverSampler,SMOTE,BorderlineSMOTE, ADASYN
from collections import Counter

In [98]:
## 오버샘플링 적용을 위한 x, y값 분류
x_train = train.drop(columns = ['buf_1000'])
y_train = train[['buf_1000']]

In [99]:
## y값 오버샘플링
smote = SMOTE(random_state = 42, k_neighbors = 5)
X_smote,y_smote = smote.fit_resample(x_train, y_train)

print('ddd %s'%Counter(y_train))
print('ddd %s'%Counter(y_smote))

ddd Counter({'buf_1000': 1})
ddd Counter({'buf_1000': 1})


In [100]:
## 오버샘플링한 x,y다시 결합
train = pd.merge(X_smote, y_smote, left_index = True, right_index = True)

In [101]:
## y값의 비율 재확인 - y 값의 비율이 일정하게 잘 샘플링 된 것을 확인 가능
train[['buf_1000']].value_counts()/len(train)

buf_1000
0           0.5
1           0.5
dtype: float64

###### pycaret 사용하여 모델 선택

In [102]:
## 모델 선택
reg = setup(train, target = 'buf_1000')

,Description,Value
0,session_id,4146
1,Target,buf_1000
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(81152, 17)"
5,Missing Values,False
6,Numeric Features,16
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='buf_1000',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_str...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [103]:
top3 = compare_models(n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dt,Decision Tree Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,0.9999,0.9999,0.068
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,0.9999,0.9999,2.036
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,0.9999,0.9999,0.073
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,0.9999,0.9999,4.265
lightgbm,Light Gradient Boosting Machine,0.9996,1.0000,0.9997,0.9995,0.9996,0.9992,0.9992,0.435
et,Extra Trees Classifier,0.9986,1.0000,1.0000,0.9972,0.9986,0.9972,0.9972,2.160
svm,SVM - Linear Kernel,0.9771,0.0000,0.9660,0.9879,0.9768,0.9542,0.9545,0.089
knn,K Neighbors Classifier,0.9768,0.9930,0.9989,0.9566,0.9773,0.9536,0.9546,1.980
lr,Logistic Regression,0.9761,0.9966,0.9686,0.9833,0.9759,0.9521,0.9522,0.357
qda,Quadratic Discriminant Analysis,0.9595,0.9891,1.0000,0.9250,0.9611,0.9189,0.9220,0.071


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=4146, splitter='best'), RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1,

In [104]:
## 하이퍼파라미터 튜닝
tuned_top3 =  [tune_model(i) for i in top3]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,0.9998,0.9998,0.9996,1.0000,0.9998,0.9996,0.9996
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,0.9998,0.9998,1.0000,0.9996,0.9998,0.9996,0.9996


INFO:logs:create_model_container: 17
INFO:logs:master_model_container: 17
INFO:logs:display_container: 5
INFO:logs:AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
                   learning_rate=0.0001, n_estimators=50, random_state=4146)
INFO:logs:tune_model() succesfully completed......................................


In [105]:
## 블랜딩해서 Voting 방식으로 앙상블
blend_top3 = blend_models(estimator_list = tuned_top3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
2,0.9998,1.0,0.9996,1.0000,0.9998,0.9996,0.9996
3,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
5,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
7,1.0000,1.0,1.0000,1.0000,1.0000,1.0000,1.0000
8,0.9998,1.0,1.0000,0.9996,0.9998,0.9996,0.9996


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 6
INFO:logs:VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=16,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.02,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                    

In [106]:
## 최종 모델
final_model_1000 = finalize_model(blend_top3) 

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=16,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.02,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
 

In [107]:
## k-fold를 통해 모델의 성능 측정을 위해서 X, Y값 분리해주기
train_Y = train[['buf_1000']]
train_X = train.drop(columns = ['buf_1000'])

In [108]:
## train_set을 집어넣어서 k-겹 교차 검증 하기
from sklearn.model_selection import cross_val_score

scores = cross_val_score(final_model_1000, # 학습한 모델
                       train_X, # X
                       train_Y, # y
                       cv=6) 
scores

array([1., 1., 1., 1., 1., 1.])

In [109]:
## 테스트 데이터셋으로 모델 성능 최종 확인
# x,y값 분리
test_Y = test[['buf_1000']]
test_X = test.drop(columns = ['buf_1000'])

In [110]:
## 인덱스 리셋
test_X.reset_index(inplace = True, drop = True)
test_Y.reset_index(inplace = True, drop = True)

In [111]:
## test셋에 예측 모델 적용
prediction = predict_model(final_model_1000, data = test_X)
prediction.reset_index(inplace = True)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=16,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.02,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
   

In [112]:
## 값이 일치하지 않는 행이 몇개 있는지 확인
prediction[prediction['Label'] != test_Y['buf_1000']]

,index,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황,Label,Score


- 수소 충전소별 1000m 버퍼에 해당하는 격자를 1로 나타낸 데이터 모델 : 추후 LPG 1000m 버퍼에 해당하는 격자들에 적용할 모델

#### LPG 반경별 그리드, 주유소명과 모델결과 도출용 데이터 생성

LPG 업체명별, 버퍼별 gid 번호 추출은 QGIS 사용

##### LPG 250 버퍼

In [179]:
## qgis에서 구한 각 lpg주유소명과 250버퍼에 해당하는 gid번호 추출한 데이터 불러오기
lpg_250_name = gpd.read_file("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/lpg충전소/lpg충전소_250/lpg충전소_250.shp", encoding="utf-8")
grid_name = list(lpg_250_name['gid'].drop_duplicates())
lpg_250_name

,업체명,종류,주소,구,동,gid,geometry
0,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바898172,"POLYGON ((989845.192 1817205.673, 989880.702 1..."
1,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바898170,"POLYGON ((989845.192 1817051.165, 989837.458 1..."
2,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바902172,"POLYGON ((990285.211 1817275.365, 990300.000 1..."
3,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바901173,"POLYGON ((990160.211 1817366.183, 990200.000 1..."
4,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바899170,"POLYGON ((989900.000 1817100.000, 990000.000 1..."
...,...,...,...,...,...,...,...
6904,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바949178,"POLYGON ((994900.000 1817900.000, 995000.000 1..."
6905,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948177,"POLYGON ((994825.244 1817766.825, 994830.499 1..."
6906,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948176,"POLYGON ((994872.990 1817619.879, 994837.480 1..."
6907,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948179,"POLYGON ((994872.990 1817913.771, 994900.000 1..."


In [180]:
## 쉬운 가공을 위해 모델 fitting용 데이터 불러오기
model_buffer_250=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer250.csv")
model_buffer_250

,Unnamed: 0,gid,area,population,buf_250,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,2,다바905261,104399.92,11.0,1,POLYGON ((127.39401232266326 36.43240119743217...,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [181]:
## 버퍼에 해당하는 gid번호에 대한 데이터만 뽑기 위해 merge 사용
model_lpg=model_buffer_250.merge(lpg_250_name,on="gid",how="left",suffixes=('', '_y'))
model_lpg

,Unnamed: 0,gid,area,population,buf_250,geometry,school_new,link_cnt,node_cnt,격자별화재건수,...,스포츠,음식,학문/교육,격자별차량현황,업체명,종류,주소,구,동,geometry_y
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,세종셀프주유소,"휘발유, 경유",대덕대로 1494,대덕구,목상동,"POLYGON ((992212.184 1828200.000, 992300.000 1..."
1,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,학하 목상셀프주유소,"휘발유, 경유",대덕대로 1500,대덕구,목상동,"POLYGON ((992232.812 1828183.663, 992264.874 1..."
2,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,...,1,31,1,23.271468,NaN,NaN,NaN,NaN,NaN,None
3,2,다바905261,104399.92,11.0,1,POLYGON ((127.39401232266326 36.43240119743217...,0.0,2,0,0.090580,...,0,8,0,23.271468,NaN,NaN,NaN,NaN,NaN,None
4,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,...,0,3,0,61.063241,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57516,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,...,0,0,0,4.528245,NaN,NaN,NaN,NaN,NaN,None
57517,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,...,0,0,0,9.380766,NaN,NaN,NaN,NaN,NaN,None
57518,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,...,0,0,0,3.266115,NaN,NaN,NaN,NaN,NaN,None
57519,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,...,0,0,0,4.885840,NaN,NaN,NaN,NaN,NaN,None


In [182]:
# 주유소값 nan값인 행 제거
model_lpg = model_lpg.dropna(subset=['업체명']).reset_index(drop=True)

주유소별로 점수화를 위한 주유소와 그리드 이름 변수 - where_lpg_gid

In [183]:
where_lpg_gid = model_lpg[['gid','업체명','주소','구','동']]
where_lpg_gid # 나중에 주유소별로 점수화를 하기 위해서 필요한 정보

,gid,업체명,주소,구,동
0,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동
1,다바922281,학하 목상셀프주유소,대덕대로 1500,대덕구,목상동
2,다바943169,재하주유소,대전 동구 동서대로 1664,동구,성남동
3,다바930151,대양주유소,우암로 4,중구,선화동
4,다바930151,중앙주유소,대종로 558,중구,선화동
...,...,...,...,...,...
6904,다바843190,GS칼텍스 정다운주유소,대전 유성구 북유성대로 51,유성구,노은1동
6905,다바912207,GS칼텍스 한빛주유소,대전 유성구 엑스포로 315,유성구,원촌동
6906,다바822167,한밭대앞셀프주유소,대전 유성구 학하서로 111,유성구,덕명동
6907,다바820180,GS칼텍스 덕명주유소,대전 유성구 현충원로 283,유성구,덕명동


In [184]:
## 주유소데이터 열 제거 - 모델적용할 데이터 생성 위해서
model_lpg=model_lpg.iloc[:,:20]

In [185]:
where_lpg_gid.reset_index(inplace=True, drop=True)
model_lpg.reset_index(inplace=True, drop=True)

In [186]:
## gid열 중복제거
model_lpg=model_lpg.iloc[model_lpg[['gid']].drop_duplicates().index,:]
model_lpg

,Unnamed: 0,gid,area,population,buf_250,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
2,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
3,19,다바930151,50751.13,1012.0,1,POLYGON ((127.42200288718107 36.33325376207827...,0.8,7,1,0.133858,1.0,0,0,0,2,0,0,1,1,48.473333
5,24,다바900178,46310.27,NaN,1,POLYGON ((127.38854091705426 36.35756886927513...,0.0,1,0,0.124706,1.0,1,1,7,6,0,1,6,1,61.063241
6,27,다바898178,42509.82,NaN,1,POLYGON ((127.38631173931851 36.35756676910772...,0.0,6,1,0.124706,1.0,0,0,2,0,0,0,4,0,61.063241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6903,54607,다바843190,NaN,NaN,0,"POLYGON ((127.32498523576082 36.3683111427302,...",0.0,0,0,0.039823,0.8,0,0,0,0,0,0,0,0,10.816523
6905,54826,다바912207,NaN,NaN,0,POLYGON ((127.40188316393488 36.38372517431598...,0.0,0,0,0.009346,0.4,0,0,0,0,0,0,0,0,14.898438
6906,54868,다바822167,NaN,NaN,0,POLYGON ((127.3016283613045 36.347539252929955...,0.0,0,0,0.017668,0.6,0,0,0,0,0,0,0,0,18.087692
6907,54904,다바820180,NaN,NaN,0,POLYGON ((127.29936940700665 36.35925549428208...,0.0,0,0,0.017668,0.8,0,0,0,0,0,0,0,0,18.087692


In [187]:
model_lpg.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_250.csv",encoding="utf-8-sig")
where_lpg_gid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_250.csv",encoding="utf-8-sig")

##### LPG 500 버퍼

In [188]:
## qgis에서 구한 각 lpg주유소명과 500버퍼에 해당하는 gid번호 추출한 데이터 불러오기
lpg_500_name = gpd.read_file("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/lpg충전소/lpg충전소_500/lpg충전소_500.shp",encoding="utf-8")
grid_name = list(lpg_500_name['gid'].drop_duplicates())
lpg_500_name

,업체명,종류,주소,구,동,gid,geometry
0,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바898167,"POLYGON ((989800.000 1816718.338, 989800.000 1..."
1,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바900176,"POLYGON ((990082.956 1817628.419, 990100.000 1..."
2,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바899176,"POLYGON ((989928.448 1817603.947, 990000.000 1..."
3,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바897167,"POLYGON ((989789.064 1816723.910, 989712.974 1..."
4,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바898166,"POLYGON ((989835.990 1816700.000, 989900.000 1..."
...,...,...,...,...,...,...,...
22389,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바952179,"POLYGON ((995200.000 1818000.000, 995300.000 1..."
22390,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바947181,"POLYGON ((994781.352 1818171.334, 994800.000 1..."
22391,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948182,"POLYGON ((994900.000 1818231.788, 994900.000 1..."
22392,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바949181,"POLYGON ((994900.000 1818200.000, 995000.000 1..."


In [189]:
## 쉬운 가공을 위해 모델 fitting용 데이터 불러오기
model_buffer_500=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer500.csv")
model_buffer_500

,Unnamed: 0,gid,area,population,buf_500,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,2,다바905261,104399.92,11.0,1,POLYGON ((127.39401232266326 36.43240119743217...,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [190]:
## 버퍼에 해당하는 gid번호에 대한 데이터만 뽑기 위해 merge 사용
model_lpg=model_buffer_500.merge(lpg_500_name,on="gid",how="left",suffixes=('', '_y'))
model_lpg

,Unnamed: 0,gid,area,population,buf_500,geometry,school_new,link_cnt,node_cnt,격자별화재건수,...,스포츠,음식,학문/교육,격자별차량현황,업체명,종류,주소,구,동,geometry_y
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,세종셀프주유소,"휘발유, 경유",대덕대로 1494,대덕구,목상동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
1,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,학하 목상셀프주유소,"휘발유, 경유",대덕대로 1500,대덕구,목상동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
2,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,...,1,31,1,23.271468,NaN,NaN,NaN,NaN,NaN,None
3,2,다바905261,104399.92,11.0,1,POLYGON ((127.39401232266326 36.43240119743217...,0.0,2,0,0.090580,...,0,8,0,23.271468,NaN,NaN,NaN,NaN,NaN,None
4,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,...,0,3,0,61.063241,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67750,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,...,0,0,0,4.528245,NaN,NaN,NaN,NaN,NaN,None
67751,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,...,0,0,0,9.380766,NaN,NaN,NaN,NaN,NaN,None
67752,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,...,0,0,0,3.266115,NaN,NaN,NaN,NaN,NaN,None
67753,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,...,0,0,0,4.885840,NaN,NaN,NaN,NaN,NaN,None


In [191]:
## 주유소값 nan값인 행 제거
model_lpg = model_lpg.dropna(subset=['업체명']).reset_index(drop=True)

주유소별로 점수화를 위한 주유소와 그리드 이름 변수 - where_lpg_gid

In [192]:
where_lpg_gid = model_lpg[['gid','업체명','주소','구','동']]
where_lpg_gid # 나중에 주유소별로 점수화를 하기 위해서 필요한 정보

,gid,업체명,주소,구,동
0,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동
1,다바922281,학하 목상셀프주유소,대덕대로 1500,대덕구,목상동
2,다바943169,재하주유소,대전 동구 동서대로 1664,동구,성남동
3,다바943169,대전SELF주유소,대전 동구 계족로 436,동구,용전동
4,다바943169,호원주유소,대전 동구 한밭대로 1315,동구,용전동
...,...,...,...,...,...
22389,다바848193,GS칼텍스 정다운주유소,대전 유성구 북유성대로 51,유성구,노은1동
22390,다바918209,강변주유소,대전 유성구 엑스포로 293,유성구,원촌동
22391,다바918209,GS칼텍스 한빛주유소,대전 유성구 엑스포로 315,유성구,원촌동
22392,다바820180,GS칼텍스 덕명주유소,대전 유성구 현충원로 283,유성구,덕명동


In [193]:
## 주유소데이터 열 제거 - 모델적용할 데이터 생성 위해서
model_lpg=model_lpg.iloc[:,:20]

In [194]:
where_lpg_gid.reset_index(inplace=True, drop=True)
model_lpg.reset_index(inplace=True, drop=True)

In [195]:
## gid열 중복제거
model_lpg=model_lpg.iloc[model_lpg[['gid']].drop_duplicates().index,:]
model_lpg

,Unnamed: 0,gid,area,population,buf_500,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
2,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
6,8,다바898167,76836.63,464.0,1,POLYGON ((127.38632615780085 36.34764983543061...,0.0,6,1,0.245161,1.0,1,7,12,14,0,0,2,1,77.129032
7,9,다바962121,70838.62,NaN,1,POLYGON ((127.45767332379349 36.30622533423838...,0.6,2,0,0.066225,1.0,0,0,4,73,0,1,20,1,43.450413
8,13,다바900176,57959.90,NaN,1,POLYGON ((127.38854348770806 36.35576579151448...,0.0,0,0,0.124706,1.0,0,0,2,22,0,0,15,0,110.086022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22388,54868,다바822167,NaN,NaN,0,POLYGON ((127.3016283613045 36.347539252929955...,0.0,0,0,0.017668,0.6,0,0,0,0,0,0,0,0,18.087692
22389,54895,다바848193,NaN,NaN,0,POLYGON ((127.33055306715453 36.37102378145104...,0.0,2,0,0.032258,0.4,0,0,0,0,0,0,0,0,10.816523
22390,54898,다바918209,NaN,NaN,0,POLYGON ((127.40857082939414 36.38553355218255...,0.0,0,0,0.009346,0.0,0,0,0,0,0,0,0,0,14.898438
22392,54904,다바820180,NaN,NaN,0,POLYGON ((127.29936940700665 36.35925549428208...,0.0,0,0,0.017668,0.8,0,0,0,0,0,0,0,0,18.087692


In [196]:
model_lpg.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_500.csv",encoding="utf-8-sig")
where_lpg_gid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_500.csv",encoding="utf-8-sig")

##### LPG 750 버퍼

In [197]:
## qgis에서 구한 각 lpg주유소명과 750버퍼에 해당하는 gid번호 추출한 데이터 불러오기
lpg_750_name = gpd.read_file("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/lpg충전소/lpg충전소_750/lpg충전소_750.shp",encoding="utf-8")
grid_name = list(lpg_250_name['gid'].drop_duplicates())
lpg_750_name

,업체명,종류,주소,구,동,gid,geometry
0,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바894175,"POLYGON ((989476.194 1817569.258, 989500.000 1..."
1,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바898167,"POLYGON ((989800.000 1816800.000, 989900.000 1..."
2,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바900176,"POLYGON ((990000.000 1817700.000, 990100.000 1..."
3,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바899176,"POLYGON ((989900.000 1817700.000, 990000.000 1..."
4,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바900178,"POLYGON ((990082.956 1817878.419, 990100.000 1..."
...,...,...,...,...,...,...,...
46654,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948182,"POLYGON ((994800.000 1818300.000, 994900.000 1..."
46655,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바954183,"POLYGON ((995500.000 1818381.783, 995500.000 1..."
46656,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바949181,"POLYGON ((994900.000 1818200.000, 995000.000 1..."
46657,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948181,"POLYGON ((994800.000 1818200.000, 994900.000 1..."


In [198]:
## 쉬운 가공을 위해 모델 fitting용 데이터 불러오기
model_buffer_750=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer750.csv")
model_buffer_750

,Unnamed: 0,gid,area,population,buf_750,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,2,다바905261,104399.92,11.0,1,POLYGON ((127.39401232266326 36.43240119743217...,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [199]:
## 버퍼에 해당하는 gid번호에 대한 데이터만 뽑기 위해 merge 사용
model_lpg=model_buffer_750.merge(lpg_750_name,on="gid",how="left",suffixes=('', '_y'))
model_lpg

,Unnamed: 0,gid,area,population,buf_750,geometry,school_new,link_cnt,node_cnt,격자별화재건수,...,스포츠,음식,학문/교육,격자별차량현황,업체명,종류,주소,구,동,geometry_y
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,세종셀프주유소,"휘발유, 경유",대덕대로 1494,대덕구,목상동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
1,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,학하 목상셀프주유소,"휘발유, 경유",대덕대로 1500,대덕구,목상동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
2,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,...,1,31,1,23.271468,NaN,NaN,NaN,NaN,NaN,None
3,2,다바905261,104399.92,11.0,1,POLYGON ((127.39401232266326 36.43240119743217...,0.0,2,0,0.090580,...,0,8,0,23.271468,NaN,NaN,NaN,NaN,NaN,None
4,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,...,0,3,0,61.063241,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,"POLYGON ((989476.194 1817569.258, 989500.000 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87664,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,...,0,0,0,4.528245,NaN,NaN,NaN,NaN,NaN,None
87665,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,...,0,0,0,9.380766,NaN,NaN,NaN,NaN,NaN,None
87666,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,...,0,0,0,3.266115,NaN,NaN,NaN,NaN,NaN,None
87667,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,...,0,0,0,4.885840,NaN,NaN,NaN,NaN,NaN,None


In [200]:
## 주유소값 nan값인 행 제거
model_lpg = model_lpg.dropna(subset=['업체명']).reset_index(drop=True)

주유소별로 점수화를 위한 주유소와 그리드 이름 변수 - where_lpg_gid

In [201]:
where_lpg_gid = model_lpg[['gid','업체명','주소','구','동']]
where_lpg_gid # 나중에 주유소별로 점수화를 하기 위해서 필요한 정보

,gid,업체명,주소,구,동
0,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동
1,다바922281,학하 목상셀프주유소,대덕대로 1500,대덕구,목상동
2,다바894175,KH에너지직영 둔산주유소,대전 서구 둔산로 139,서구,둔산동
3,다바894175,현대오일뱅크직영 타임셀프주유소,대전 서구 대덕대로 188,서구,둔산동
4,다바894175,GS칼텍스 우리동네주유소,대전 서구 청사로 224,서구,둔산동
...,...,...,...,...,...
46654,다바918209,회덕IC충전소,갑선도시고속도로 1084,대덕구,신대동
46655,다바918209,씨앤에스유통 회덕IC주유소,갑선도시고속도로 1084,대덕구,신대동
46656,다바820180,온천주유소,대전 유성구 현충원로 342,유성구,구암동
46657,다바820180,GS칼텍스 덕명주유소,대전 유성구 현충원로 283,유성구,덕명동


In [202]:
# 주유소데이터 열 제거 - 모델적용할 데이터 생성 위해서
model_lpg=model_lpg.iloc[:,:20]

In [203]:
where_lpg_gid.reset_index(inplace=True, drop=True)
model_lpg.reset_index(inplace=True, drop=True)

In [204]:
# gid열 중복제거
model_lpg=model_lpg.iloc[model_lpg[['gid']].drop_duplicates().index,:]
model_lpg

,Unnamed: 0,gid,area,population,buf_750,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
2,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
5,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
17,5,다바906198,91368.63,565.0,1,POLYGON ((127.39520427564275 36.37560567182181...,0.0,2,0,0.025157,1.0,0,3,10,4,0,0,31,0,4.528245
19,7,다바915251,81601.11,8.0,1,"POLYGON ((127.4051798693703 36.42339529431997,...",0.0,2,0,0.019900,1.0,1,0,4,16,0,0,3,0,23.271468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46646,54868,다바822167,NaN,NaN,0,POLYGON ((127.3016283613045 36.347539252929955...,0.0,0,0,0.017668,0.6,0,0,0,0,0,0,0,0,18.087692
46647,54895,다바848193,NaN,NaN,0,POLYGON ((127.33055306715453 36.37102378145104...,0.0,2,0,0.032258,0.4,0,0,0,0,0,0,0,0,10.816523
46650,54898,다바918209,NaN,NaN,0,POLYGON ((127.40857082939414 36.38553355218255...,0.0,0,0,0.009346,0.0,0,0,0,0,0,0,0,0,14.898438
46656,54904,다바820180,NaN,NaN,0,POLYGON ((127.29936940700665 36.35925549428208...,0.0,0,0,0.017668,0.8,0,0,0,0,0,0,0,0,18.087692


In [205]:
model_lpg.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_750.csv",encoding="utf-8-sig")
where_lpg_gid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_750.csv",encoding="utf-8-sig")

##### LPG 1000 버퍼

In [206]:
## qgis에서 구한 각 lpg주유소명과 1000버퍼에 해당하는 gid번호 추출한 데이터 불러오기
lpg_1000_name = gpd.read_file("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/lpg충전소/lpg충전소_1000/lpg충전소_1000.shp",encoding="utf-8")
grid_name = list(lpg_250_name['gid'].drop_duplicates())
lpg_1000_name

,업체명,종류,주소,구,동,gid,geometry
0,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바894175,"POLYGON ((989400.000 1817600.000, 989500.000 1..."
1,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바898167,"POLYGON ((989800.000 1816800.000, 989900.000 1..."
2,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바900176,"POLYGON ((990000.000 1817700.000, 990100.000 1..."
3,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바899176,"POLYGON ((989900.000 1817700.000, 990000.000 1..."
4,KH에너지직영 둔산주유소,"휘발유,경유",대전 서구 둔산로 139,서구,둔산동,다바900178,"POLYGON ((990000.000 1817900.000, 990100.000 1..."
...,...,...,...,...,...,...,...
79608,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바949181,"POLYGON ((994900.000 1818200.000, 995000.000 1..."
79609,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바952187,"POLYGON ((995300.000 1818731.227, 995300.000 1..."
79610,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948181,"POLYGON ((994800.000 1818200.000, 994900.000 1..."
79611,동양주유소,"휘발유, 경유",동서대로 1811,대덕구,송촌동,다바948183,"POLYGON ((994800.000 1818400.000, 994900.000 1..."


In [207]:
## 쉬운 가공을 위해 모델 fitting용 데이터 불러오기
model_buffer_1000=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/All_buffer1000.csv")
model_buffer_1000

,Unnamed: 0,gid,area,population,buf_1000,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,1.0,0,0,2,78,1,1,31,1,23.271468
2,2,다바905261,104399.92,11.0,1,POLYGON ((127.39401232266326 36.43240119743217...,0.0,2,0,0.090580,1.0,0,2,4,9,0,0,8,0,23.271468
3,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
4,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,0.4,0,0,0,0,0,0,0,0,4.528245
54908,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,0.4,0,0,0,0,0,0,0,0,9.380766
54909,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,0.8,0,0,0,0,0,0,0,0,3.266115
54910,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,0.0,0,0,0,0,0,0,0,0,4.885840


In [208]:
## 버퍼에 해당하는 gid번호에 대한 데이터만 뽑기 위해 merge 사용
model_lpg=model_buffer_1000.merge(lpg_1000_name,on="gid",how="left",suffixes=('', '_y'))
model_lpg

,Unnamed: 0,gid,area,population,buf_1000,geometry,school_new,link_cnt,node_cnt,격자별화재건수,...,스포츠,음식,학문/교육,격자별차량현황,업체명,종류,주소,구,동,geometry_y
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,공단LPG충전소,LPG,대덕대로 1408,대덕구,신일동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
1,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,현대오일뱅크직영 4공단주유소,"휘발유, 경유",대덕대로 1412,대덕구,신일동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
2,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,세종셀프주유소,"휘발유, 경유",대덕대로 1494,대덕구,목상동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
3,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,...,0,0,0,8.962712,학하 목상셀프주유소,"휘발유, 경유",대덕대로 1500,대덕구,목상동,"POLYGON ((992200.000 1828200.000, 992300.000 1..."
4,1,다바909251,118444.81,NaN,1,POLYGON ((127.3984866928581 36.423389791632225...,0.0,0,0,0.019900,...,1,31,1,23.271468,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117278,54907,다바888214,NaN,NaN,0,POLYGON ((127.37511398610806 36.39001093526398...,0.0,0,0,0.025157,...,0,0,0,4.528245,NaN,NaN,NaN,NaN,NaN,None
117279,54908,다바827236,NaN,NaN,0,POLYGON ((127.30704697381218 36.40975409072774...,0.0,0,0,0.007212,...,0,0,0,9.380766,NaN,NaN,NaN,NaN,NaN,None
117280,54909,다바827134,NaN,NaN,0,POLYGON ((127.30727386191882 36.31779758247113...,0.0,2,0,0.011848,...,0,0,0,3.266115,NaN,NaN,NaN,NaN,NaN,None
117281,54910,다바898317,NaN,NaN,0,POLYGON ((127.38612893353715 36.48287932693206...,0.0,0,0,0.000000,...,0,0,0,4.885840,NaN,NaN,NaN,NaN,NaN,None


In [209]:
## 주유소값 nan값인 행 제거
model_lpg = model_lpg.dropna(subset=['업체명']).reset_index(drop=True)

주유소별로 점수화를 위한 주유소와 그리드 이름 변수 - where_lpg_gid

In [210]:
where_lpg_gid = model_lpg[['gid','업체명','주소','구','동']]
where_lpg_gid # 나중에 주유소별로 점수화를 하기 위해서 필요한 정보

,gid,업체명,주소,구,동
0,다바922281,공단LPG충전소,대덕대로 1408,대덕구,신일동
1,다바922281,현대오일뱅크직영 4공단주유소,대덕대로 1412,대덕구,신일동
2,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동
3,다바922281,학하 목상셀프주유소,대덕대로 1500,대덕구,목상동
4,다바894175,KH에너지직영 둔산주유소,대전 서구 둔산로 139,서구,둔산동
...,...,...,...,...,...
79608,다바820180,온천주유소,대전 유성구 현충원로 342,유성구,구암동
79609,다바820180,GS칼텍스 덕명주유소,대전 유성구 현충원로 283,유성구,덕명동
79610,다바820180,계룡주유소,대전 유성구 현충원로 177,유성구,갑동
79611,다바820180,SK행복충전 유성충전소,대전 유성구 현충원로 168,유성구,갑동


In [211]:
# 주유소데이터 열 제거 - 모델적용할 데이터 생성 위해서
model_lpg=model_lpg.iloc[:,:20]

In [212]:
where_lpg_gid.reset_index(inplace=True, drop=True)
model_lpg.reset_index(inplace=True, drop=True)

In [213]:
# gid열 중복제거
model_lpg=model_lpg.iloc[model_lpg[['gid']].drop_duplicates().index,:]
model_lpg

,Unnamed: 0,gid,area,population,buf_1000,geometry,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
4,3,다바894175,102749.66,NaN,1,POLYGON ((127.38185747179782 36.35485782779705...,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
8,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
25,5,다바906198,91368.63,565.0,1,POLYGON ((127.39520427564275 36.37560567182181...,0.0,2,0,0.025157,1.0,0,3,10,4,0,0,31,0,4.528245
28,6,다바838156,87955.33,0.0,1,POLYGON ((127.31948232460852 36.33765060156227...,0.0,0,0,0.015228,1.0,0,0,3,11,0,0,7,0,18.087692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79594,54868,다바822167,NaN,NaN,0,POLYGON ((127.3016283613045 36.347539252929955...,0.0,0,0,0.017668,0.6,0,0,0,0,0,0,0,0,18.087692
79595,54895,다바848193,NaN,NaN,0,POLYGON ((127.33055306715453 36.37102378145104...,0.0,2,0,0.032258,0.4,0,0,0,0,0,0,0,0,10.816523
79599,54898,다바918209,NaN,NaN,0,POLYGON ((127.40857082939414 36.38553355218255...,0.0,0,0,0.009346,0.0,0,0,0,0,0,0,0,0,14.898438
79608,54904,다바820180,NaN,NaN,0,POLYGON ((127.29936940700665 36.35925549428208...,0.0,0,0,0.017668,0.8,0,0,0,0,0,0,0,0,18.087692


In [214]:
model_lpg.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_1000.csv",encoding="utf-8-sig")
where_lpg_gid.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_1000.csv",encoding="utf-8-sig")

#### LPG 충전소의 버퍼에 해당하는 격자별 모델 적용

##### 250 버퍼

In [215]:
model_doit=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_250.csv")

In [216]:
model_doit

,Unnamed: 0,Unnamed: 0.1,gid,area,population,buf_250,geometry,school_new,link_cnt,node_cnt,...,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,0,다바922281,125401.97,NaN,1,POLYGON ((127.41295838161777 36.45044706212179...,0.0,0,0,...,0.4,0,0,0,0,0,0,0,0,8.962712
1,2,4,다바943169,95862.58,NaN,1,"POLYGON ((127.4364748788022 36.34949009937457,...",0.2,1,0,...,1.0,0,0,8,70,0,0,44,0,70.595041
2,3,19,다바930151,50751.13,1012.0,1,POLYGON ((127.42200288718107 36.33325376207827...,0.8,7,1,...,1.0,0,0,0,2,0,0,1,1,48.473333
3,5,24,다바900178,46310.27,NaN,1,POLYGON ((127.38854091705426 36.35756886927513...,0.0,1,0,...,1.0,1,1,7,6,0,1,6,1,61.063241
4,6,27,다바898178,42509.82,NaN,1,POLYGON ((127.38631173931851 36.35756676910772...,0.0,6,1,...,1.0,0,0,2,0,0,0,4,0,61.063241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4295,6903,54607,다바843190,NaN,NaN,0,"POLYGON ((127.32498523576082 36.3683111427302,...",0.0,0,0,...,0.8,0,0,0,0,0,0,0,0,10.816523
4296,6905,54826,다바912207,NaN,NaN,0,POLYGON ((127.40188316393488 36.38372517431598...,0.0,0,0,...,0.4,0,0,0,0,0,0,0,0,14.898438
4297,6906,54868,다바822167,NaN,NaN,0,POLYGON ((127.3016283613045 36.347539252929955...,0.0,0,0,...,0.6,0,0,0,0,0,0,0,0,18.087692
4298,6907,54904,다바820180,NaN,NaN,0,POLYGON ((127.29936940700665 36.35925549428208...,0.0,0,0,...,0.8,0,0,0,0,0,0,0,0,18.087692


###### 모델 적용 및 전처리 결과

In [217]:
## null값 0으로 대체
model_doit.fillna(0, inplace = True)
model_doit.isnull().sum()

Unnamed: 0      0
Unnamed: 0.1    0
gid             0
area            0
population      0
buf_250         0
geometry        0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [218]:
## gid, geometry변수 제거 및 gid값 저장
gid = model_doit[['gid']]
model_doit.drop(['gid', 'geometry'], axis = 1, inplace = True)

In [219]:
## 범주형변수이므로 정규화에서 제외시켜줌
model_doit_target = model_doit[['buf_250']]
model_doit_numeric = model_doit.drop(columns=['buf_250'], axis=1)
model_doit_numeric.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,2,4,95862.58,0.0,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
2,3,19,50751.13,1012.0,0.8,7,1,0.133858,1.0,0,0,0,2,0,0,1,1,48.473333


In [220]:
## 데이터 정규화
scaler = MinMaxScaler()
model_doit_numeric_scale = scaler.fit_transform(model_doit_numeric)

In [221]:
## 정규화 데이터 DF로 변환
model_doit_scale = pd.DataFrame(
    model_doit_numeric_scale,
    columns = list(model_doit_numeric.columns))
model_doit_scale.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.162037,0.4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.073247
1,0.000290,0.000073,0.764442,0.000000,0.2,0.045455,0.0,0.404959,1.0,0.0,0.0,0.275862,0.328638,0.0,0.0,0.830189,0.000000,0.588346
2,0.000434,0.000346,0.404708,0.834295,0.8,0.318182,0.2,0.312336,1.0,0.0,0.0,0.000000,0.009390,0.0,0.0,0.018868,0.032258,0.403461


In [222]:
## 모델 적용
prediction = predict_model(final_model_250, data=model_doit_scale)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=5,
                                                     max_features='sqrt',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.1,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=5,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',


In [223]:
## 결과값이 어떤 격자에서 일어났는지 알기 위해서 gid컬럼 추가
prediction['gid']=gid

In [224]:
## gid별 모델 적용 결과
result_250=prediction[['gid','Label']]

##### 500버퍼

In [225]:
model_doit=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_500.csv")

###### 모델적용 전 전처리 및 결과

In [226]:
## null값 0으로 대체
model_doit.fillna(0, inplace = True)
model_doit.isnull().sum()

Unnamed: 0      0
Unnamed: 0.1    0
gid             0
area            0
population      0
buf_500         0
geometry        0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [227]:
## gid, geometry변수 제거 및 gid값 저장
gid = model_doit[['gid']]
model_doit.drop(['gid', 'geometry'], axis = 1, inplace = True)

In [228]:
## 범주형변수이므로 정규화에서 제외시켜줌
model_doit_target = model_doit[['buf_500']]
model_doit_numeric = model_doit.drop(columns=['buf_500'], axis=1)
model_doit_numeric.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,2,4,95862.58,0.0,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041
2,6,8,76836.63,464.0,0.0,6,1,0.245161,1.0,1,7,12,14,0,0,2,1,77.129032


In [229]:
## 데이터 정규화
scaler = MinMaxScaler()
model_doit_numeric_scale = scaler.fit_transform(model_doit_numeric)

In [230]:
## 정규화 데이터 DF로 변환
model_doit_scale = pd.DataFrame(
    model_doit_numeric_scale,
    columns = list(model_doit_numeric.columns))
model_doit_scale.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.145202,0.4,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.056514
1,0.000089,0.000073,0.764442,0.000000,0.2,0.045455,0.0,0.362885,1.0,0.000000,0.000,0.131148,0.186667,0.0,0.0,0.814815,0.00000,0.445133
2,0.000268,0.000146,0.612723,0.169715,0.0,0.272727,0.2,0.512610,1.0,0.083333,0.875,0.196721,0.037333,0.0,0.0,0.037037,0.03125,0.486332


In [231]:
## 모델 적용
prediction = predict_model(final_model_500, data=model_doit_scale)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=15,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.005,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=5,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
  

In [232]:
## 결과값이 어떤 격자에서 일어났는지 알기 위해서 gid컬럼 추가
prediction['gid']=gid

In [233]:
## gid별 모델 적용 결과
result_500=prediction[['gid','Label']]

##### 750 버퍼

In [234]:
model_doit=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_750.csv")

###### 모델적용 전 전처리 및 결과

In [235]:
## null값 0으로 대체
model_doit.fillna(0, inplace = True)
model_doit.isnull().sum()

Unnamed: 0      0
Unnamed: 0.1    0
gid             0
area            0
population      0
buf_750         0
geometry        0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [236]:
## gid, geometry변수 제거 및 gid값 저장
gid = model_doit[['gid']]
model_doit.drop(['gid', 'geometry'], axis = 1, inplace = True)

In [237]:
## 범주형변수이므로 정규화에서 제외시켜줌
model_doit_target = model_doit[['buf_750']]
model_doit_numeric = model_doit.drop(columns=['buf_750'], axis=1)
model_doit_numeric.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,2,3,102749.66,0.0,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
2,5,4,95862.58,0.0,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041


In [238]:
## 데이터 정규화
scaler = MinMaxScaler()
model_doit_numeric_scale = scaler.fit_transform(model_doit_numeric)

In [239]:
## 정규화 데이터 DF로 변환
model_doit_scale = pd.DataFrame(
    model_doit_numeric_scale,
    columns = list(model_doit_numeric.columns))
model_doit_scale.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.145202,0.4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.056514
1,0.000043,0.000055,0.819362,0.0,0.8,0.136364,0.0,0.260749,1.0,0.0,0.0,0.016393,0.008000,0.0,0.0,0.046875,0.0,0.385031
2,0.000107,0.000073,0.764442,0.0,0.2,0.045455,0.0,0.362885,1.0,0.0,0.0,0.131148,0.186667,0.0,0.0,0.687500,0.0,0.445133


In [240]:
## 모델 적용
prediction = predict_model(final_model_750, data=model_doit_scale)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='entropy',
                                                     max_depth=9,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.3,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
  

In [241]:
## 결과값이 어떤 격자에서 일어났는지 알기 위해서 gid컬럼 추가
prediction['gid']=gid

In [242]:
## gid별 모델 적용 결과
result_750=prediction[['gid','Label']]

##### 1000 버퍼

In [243]:
model_doit=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/model_doit_1000.csv")

###### 모델적용 전 전처리 및 결과

In [244]:
## null값 0으로 대체
model_doit.fillna(0, inplace = True)
model_doit.isnull().sum()

Unnamed: 0      0
Unnamed: 0.1    0
gid             0
area            0
population      0
buf_1000        0
geometry        0
school_new      0
link_cnt        0
node_cnt        0
격자별화재건수         0
business_new    0
관광/여가/오락        0
부동산             0
생활서비스           0
소매              0
숙박              0
스포츠             0
음식              0
학문/교육           0
격자별차량현황         0
dtype: int64

In [245]:
## gid, geometry변수 제거 및 gid값 저장
gid = model_doit[['gid']]
model_doit.drop(['gid', 'geometry'], axis = 1, inplace = True)

In [246]:
## 범주형변수이므로 정규화에서 제외시켜줌
model_doit_target = model_doit[['buf_1000']]
model_doit_numeric = model_doit.drop(columns=['buf_1000'], axis=1)
model_doit_numeric.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0,0,125401.97,0.0,0.0,0,0,0.069444,0.4,0,0,0,0,0,0,0,0,8.962712
1,4,3,102749.66,0.0,0.8,3,0,0.124706,1.0,0,0,1,3,0,0,3,0,61.063241
2,8,4,95862.58,0.0,0.2,1,0,0.173554,1.0,0,0,8,70,0,0,44,0,70.595041


In [247]:
## 데이터 정규화
scaler = MinMaxScaler()
model_doit_numeric_scale = scaler.fit_transform(model_doit_numeric)

In [248]:
## 정규화 데이터 DF로 변환
model_doit_scale = pd.DataFrame(
    model_doit_numeric_scale,
    columns = list(model_doit_numeric.columns))
model_doit_scale.head(3)

,Unnamed: 0,Unnamed: 0.1,area,population,school_new,link_cnt,node_cnt,격자별화재건수,business_new,관광/여가/오락,부동산,생활서비스,소매,숙박,스포츠,음식,학문/교육,격자별차량현황
0,0.00000,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.145202,0.4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.056514
1,0.00005,0.000055,0.819362,0.0,0.8,0.136364,0.0,0.260749,1.0,0.0,0.0,0.016393,0.008000,0.0,0.0,0.046875,0.0,0.385031
2,0.00010,0.000073,0.764442,0.0,0.2,0.045455,0.0,0.362885,1.0,0.0,0.0,0.131148,0.186667,0.0,0.0,0.687500,0.0,0.445133


In [249]:
## 모델 적용
prediction = predict_model(final_model_1000, data=model_doit_scale)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('dt',
                              DecisionTreeClassifier(ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=16,
                                                     max_features=1.0,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.02,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=3,
                                                     min_samples_split=10,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
   

In [250]:
## 결과값이 어떤 격자에서 일어났는지 알기 위해서 gid컬럼 추가
prediction['gid']=gid

In [251]:
## gid별 모델 적용 결과
result_1000=prediction[['gid','Label']]

#### 주유소별 점수화

주유소버퍼의 그리드와 주유소명이 있는 데이터(gidlpgname_250,500,750,1000)를 불러와서 위의 결과를 붙혀줌

주유소별 1의 갯수로 수소충전소를 입점하기 좋은 lpg충전소를 선택하기 위함.

##### 250 버퍼

In [252]:
## lpg충전소의 250버퍼에 해당하는 그리드데이터 가져오기
gidlpgname=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_250.csv")
gidlpgname.head(3)

,Unnamed: 0,gid,업체명,주소,구,동
0,0,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동
1,1,다바922281,학하 목상셀프주유소,대덕대로 1500,대덕구,목상동
2,2,다바943169,재하주유소,대전 동구 동서대로 1664,동구,성남동


In [253]:
## merge - 해당 그리드가 어떤 주유소에 있는지에 대해 알기 위함
realResult=gidlpgname.merge(result_250,on="gid",how="left")

In [254]:
## lpg주유소의 그리드별로 1이라고 분류한 것의 비율이 어느정도인지에 대한 스코어링 하기. - 최종 스코어링
realResult=realResult.groupby(['업체명','구']).mean().sort_values(by='Label',ascending=False)
realResult

,,Unnamed: 0,Label
업체명,구,,
삼정주유소,서구,334.366667,0.133333
중도석유 대덕주유소,서구,1424.750000,0.125000
GS칼텍스 우리동네주유소,서구,3894.125000,0.125000
GS칼텍스 유성온천주유소,유성구,1622.781250,0.093750
대양주유소,중구,2006.357143,0.071429
...,...,...,...
동아주유소,동구,3596.500000,0.000000
동양주유소,대덕구,2500.354839,0.000000
동일석유 삼영충전소,중구,4224.866667,0.000000


In [255]:
realResult.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_250.csv",encoding='utf-8-sig')

##### 500 버퍼

In [256]:
## lpg충전소의 500버퍼에 해당하는 그리드데이터 가져오기
gidlpgname=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_500.csv")
gidlpgname.head(3)

,Unnamed: 0,gid,업체명,주소,구,동
0,0,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동
1,1,다바922281,학하 목상셀프주유소,대덕대로 1500,대덕구,목상동
2,2,다바943169,재하주유소,대전 동구 동서대로 1664,동구,성남동


In [257]:
## merge - 해당 그리드가 어떤 주유소에 있는지에 대해 알기 위함
realResult=gidlpgname.merge(result_500,on="gid",how="left")

In [258]:
## lpg주유소의 그리드별로 1이라고 분류한 것의 비율이 어느정도인지에 대한 스코어링 하기. - 최종 스코어링
realResult=realResult.groupby(['업체명','구']).mean().sort_values(by='Label',ascending=False)
realResult

,,Unnamed: 0,Label
업체명,구,,
중도석유 대덕주유소,서구,6653.030928,0.278351
KH에너지직영 둔산주유소,서구,5184.214286,0.265306
중도석유 월드컵주유소,서구,4339.550000,0.220000
극동유화 한밭주유소,서구,4355.787879,0.202020
GS칼텍스 우리동네주유소,서구,12692.052083,0.197917
...,...,...,...
명품주유소,서구,10775.270833,0.000000
매봉주유소,동구,10147.292929,0.000000
맘모스충전소,중구,12532.676768,0.000000


In [259]:
realResult.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_500.csv",encoding='utf-8-sig')

##### 750 버퍼

In [260]:
## lpg충전소의 750버퍼에 해당하는 그리드데이터 가져오기
gidlpgname=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_750.csv")
gidlpgname.head(3)

,Unnamed: 0,gid,업체명,주소,구,동
0,0,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동
1,1,다바922281,학하 목상셀프주유소,대덕대로 1500,대덕구,목상동
2,2,다바894175,KH에너지직영 둔산주유소,대전 서구 둔산로 139,서구,둔산동


In [261]:
## merge - 해당 그리드가 어떤 주유소에 있는지에 대해 알기 위함
realResult=gidlpgname.merge(result_750,on="gid",how="left")

In [262]:
## lpg주유소의 그리드별로 1이라고 분류한 것의 비율이 어느정도인지에 대한 스코어링 하기. - 최종 스코어링
realResult=realResult.groupby(['업체명','구']).mean().sort_values(by='Label',ascending=False)
realResult

,,Unnamed: 0,Label
업체명,구,,
KH에너지직영 둔산주유소,서구,12081.717073,0.443902
중도석유 대덕주유소,서구,16845.028846,0.408654
GS칼텍스 우리동네주유소,서구,23482.287081,0.349282
현대오일뱅크직영 타임셀프주유소,서구,10240.813725,0.333333
중도석유 월드컵주유소,서구,10349.764706,0.264706
...,...,...,...
대성주유소,유성구,33258.348039,0.000000
동학사충전소,유성구,40582.258537,0.000000
동대전셀프주유소,대덕구,33422.724138,0.000000


In [263]:
realResult.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_750.csv",encoding='utf-8-sig')

##### 1000 버퍼

In [264]:
## lpg충전소의 750버퍼에 해당하는 그리드데이터 가져오기
gidlpgname=pd.read_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/gidlpgname_1000.csv")
gidlpgname.head(3)

,Unnamed: 0,gid,업체명,주소,구,동
0,0,다바922281,공단LPG충전소,대덕대로 1408,대덕구,신일동
1,1,다바922281,현대오일뱅크직영 4공단주유소,대덕대로 1412,대덕구,신일동
2,2,다바922281,세종셀프주유소,대덕대로 1494,대덕구,목상동


In [265]:
## merge - 해당 그리드가 어떤 주유소에 있는지에 대해 알기 위함
realResult=gidlpgname.merge(result_1000,on="gid",how="left")

In [266]:
## lpg주유소의 그리드별로 1이라고 분류한 것의 비율이 어느정도인지에 대한 스코어링 하기. - 최종 스코어링
realResult=realResult.groupby(['업체명','구']).mean().sort_values(by='Label',ascending=False)
realResult

,,Unnamed: 0,Label
업체명,구,,
KH에너지직영 둔산주유소,서구,19736.468571,0.597143
중도석유 대덕주유소,서구,33271.337176,0.458213
GS칼텍스 우리동네주유소,서구,37914.084058,0.452174
현대오일뱅크직영 타임셀프주유소,서구,20168.549575,0.444759
중도석유 월드컵주유소,서구,17597.606232,0.419263
...,...,...,...
우정주유소,동구,53855.277620,0.008499
기성농업주유소,서구,64357.257880,0.002865
고향주유소,동구,54335.264205,0.002841


In [267]:
realResult.to_csv("/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_1000.csv",encoding='utf-8-sig')

#### 최종 결과 도출

##### 데이터 불러오기

In [284]:
## 상위 30개 결과 저장
realResult250 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_250.csv').head(30)
realResult500 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_500.csv').head(30)
realResult750 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_750.csv').head(30)
realResult1000 = pd.read_csv('/content/drive/MyDrive/2022 데이터청년캠퍼스/데이터/최종데이터/realrealResult_1000.csv').head(30)

In [285]:
## 필요없는 열 삭제
realResult250.drop(['Unnamed: 0'], axis = 1, inplace = True)
realResult500.drop(['Unnamed: 0'], axis = 1, inplace = True)
realResult750.drop(['Unnamed: 0'], axis = 1, inplace = True)
realResult1000.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [286]:
a1 = realResult1000.merge(realResult750, on = ['업체명', '구'])
a1.rename(columns = {'Label_x':'buffer_1000', 'Label_y':'buffer_750'}, inplace = True)
a2 = a1.merge(realResult500, on = ['업체명', '구'])
a3 = a2.merge(realResult250, on = ['업체명', '구'])
a3.rename(columns = {'Label_x':'buffer_500', 'Label_y':'buffer_250'}, inplace = True)

In [287]:
a3

,업체명,구,buffer_1000,buffer_750,buffer_500,buffer_250
0,KH에너지직영 둔산주유소,서구,0.597143,0.443902,0.265306,0.033333
1,중도석유 대덕주유소,서구,0.458213,0.408654,0.278351,0.125000
2,GS칼텍스 우리동네주유소,서구,0.452174,0.349282,0.197917,0.125000
3,현대오일뱅크직영 타임셀프주유소,서구,0.444759,0.333333,0.170000,0.033333
4,중도석유 월드컵주유소,서구,0.419263,0.264706,0.220000,0.032258
5,극동유화 한밭주유소,서구,0.407932,0.254902,0.202020,0.066667
6,삼정주유소,서구,0.273504,0.241379,0.161616,0.133333
7,구봉산셀프주유소,서구,0.269886,0.147783,0.070707,0.033333
8,GS칼텍스 유성온천주유소,유성구,0.265130,0.201923,0.168421,0.093750
9,중도가스 서대전충전소,중구,0.247839,0.167464,0.164948,0.031250
